#### PROGRESS: Have been able to successfully access cricclubs and get the runs, ball, players and commentary data. I have also managed to convert that data into a csv format which is similar to cricsheet. Also, I got all the information about the game, such as the toss, winner, teams, event name, etc.
#### HOWEVER: The data is not 100% accurate - there are some balls where I haven't been able to access the actual data, example when players retire. 
##### Other additions: Full names of Batters and Bowlers + Text analysis using ML on the commentary data
##### Here is the link to the ChatGPT chat that helped me - https://chat.openai.com/share/6a1cb0cc-eb30-4c1f-b9a2-e4cb1fade473

### IMPORTS

In [1]:
import pandas as pd
import re
import numpy as np
import requests
from bs4 import BeautifulSoup

## THE ACTUAL SCRAPING

In [45]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
url = 'https://www.cricclubs.com/Tanzania/ballbyball.do?matchId=2001&clubId=7605'
url2 = 'https://www.cricclubs.com/Tanzania/fullScorecard.do?matchId=2121&clubId=7605'

## 1. Ball-by-Ball (innings_data)

### Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_ball_by_ball_data(match_id):
    url = f'https://www.cricclubs.com/Tanzania/ballbyball.do?matchId={match_id}&clubId=7605'
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')

    batting_team_elements = soup.find_all('li', id=lambda x: x and x.startswith('ballByBallTeamTab'))
    batting_teams = [element.find('a').text.strip() for element in batting_team_elements]
    inning_numbers = [i + 1 for i in range(len(batting_teams))]
    team_inning_mapping = dict(zip(batting_teams, inning_numbers))

    elements = soup.find_all('ul', class_='list-inline bbb-row')

    over_numbers = []
    runs_obtained = []
    batter_and_bowler_names = []
    additional_infos = []

    for element in elements:
        over_number_element = element.find('span', class_='ov')
        runs_element = element.find('span', class_='runs')
        zero_element = element.find('span', class_='zero')
        wicket_element = element.find('span', class_='wicket')
        batter_and_bowler_element = element.find('li', class_='col3')
        additional_info_element = element.find('span', class_='hidden-phone')

        if over_number_element:
            over_numbers.append(over_number_element.text.strip())
        else:
            over_numbers.append("N/A")

        if runs_element:
            runs_obtained.append(runs_element.text.strip())
        else:
            if zero_element:
                runs_obtained.append("0")
            elif wicket_element:
                runs_obtained.append(wicket_element.text.strip())
            else:
                runs_obtained.append("N/A")

        if batter_and_bowler_element:
            batter_and_bowler_names.append(batter_and_bowler_element.text.strip())
        else:
            batter_and_bowler_names.append("N/A")

        if additional_info_element:
            additional_infos.append(additional_info_element.text.strip())
        else:
            additional_infos.append("N/A")

    trial_df = pd.DataFrame({
        'Over Number': over_numbers,
        'Runs Obtained': runs_obtained,
        'Batter and Bowler Names': batter_and_bowler_names,
        'Additional Information': additional_infos
    })

    trial_df['Match ID'] = match_id

    return trial_df

In [2]:
# Scrape data for the current match ID
ball_df = scrape_ball_by_ball_data(2400)

In [3]:
# Initialize an empty list to store all scraped data
all_data = []

# Initialize empty lists to store successful and failed match IDs
success_match_ids = []
failed_match_ids = []

# Initialize fail count
fail_count = 0

# Loop through the match IDs
for i in range(1, 2300):
    # Convert the match ID to the specified format
    if i < 10:
        match_id = f'0{i}'
    elif 10 <= i < 100:
        match_id = f'{i}'
    elif 100 <= i < 1000:
        match_id = f'{i}'
    else:
        match_id = f'{i}'

    # Scrape data for the current match ID
    match_data = scrape_ball_by_ball_data(match_id)

    # Check if the match data is None (indicating a failed request)
    if match_data is None:
        # Increment the fail counter
        fail_count += 1
        # Append the failed match ID to the list of failed match IDs
        print(f"No data for Match_ID: {match_id}")
        # If the fail count is more than 5, stop the loop
        if fail_count > 5:
            break
    else:
        # Reset the fail counter
        fail_count = 0
        # Append the successful match ID to the list of successful match IDs
        print(f"Successfully scraped data for Match_ID: {match_id}")
        # Append the match data to the list of all data
        all_data.append(match_data)

# Concatenate all scraped data into one DataFrame
ball_df = pd.concat(all_data, ignore_index=True)

# Display the final DataFrame
print(ball_df.head())

Successfully scraped data for Match_ID: 01
Successfully scraped data for Match_ID: 02


KeyboardInterrupt: 

In [ ]:
ball_df.to_csv("cricclubs_ball_by_ball.csv", index=False)

In [20]:
ball_df = pd.read_csv("cricclubs_ball_by_ball.csv")

### Data Preparation and Cleaning

In [3]:
# Separate into two DataFrames based on the condition
df_with_runs = ball_df[~pd.isna(ball_df['Runs Obtained'])]  # '~' operator negates the condition
df_without_runs = ball_df[pd.isna(ball_df['Over Number'])]

# Print the resulting DataFrames
print("DataFrame with Runs:")
print(df_with_runs.tail())
print("\nDataFrame without Runs:")
print(df_without_runs.head())

DataFrame with Runs:
    Over Number Runs Obtained                   Batter and Bowler Names  \
369        10.3             0                A Hafidh to B Fredy, 0 run   
370        10.4             2               A Hafidh to B Fredy, 2 runs   
371        10.5             0                A Hafidh to B Fredy, 0 run   
372        10.6             0                A Hafidh to B Fredy, 0 run   
373                       N/A  Interruption occurred - 29 overs reduced   

    Additional Information  Match ID  
369                2:25 PM      2400  
370                2:25 PM      2400  
371                2:26 PM      2400  
372                2:26 PM      2400  
373                3:41 PM      2400  

DataFrame without Runs:
Empty DataFrame
Columns: [Over Number, Runs Obtained, Batter and Bowler Names, Additional Information, Match ID]
Index: []


In [4]:
# Check if the 'Over Number' column contains NaN values
nan_mask = pd.isna(df_with_runs['Over Number'])

# Filter out the rows with NaN values in the 'Over Number' column
df_with_runs = df_with_runs[~nan_mask]

# Apply the rest of the code for data extraction
ball_by_ball_df = df_with_runs.copy()

### Team List Method

In [5]:
# Drop rows with NaN values in the 'Batter and Bowler Names' column
df_without_runs.dropna(subset=['Batter and Bowler Names'], inplace=True)

# Extract rows where the playing squad of each team is revealed
playing_squad_rows = df_without_runs[df_without_runs['Batter and Bowler Names'].str.contains('Players:')]

# Make a separate DataFrame for the playing squad rows
playing_squad_df = playing_squad_rows.copy()

playing_squad_df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_16740\3271847602.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_runs.dropna(subset=['Batter and Bowler Names'], inplace=True)


,Over Number,Runs Obtained,Batter and Bowler Names,Additional Information,Match ID


In [6]:
def extract_players_and_captain(player_string):
    # Initialize an empty dictionary to store the results
    result = {'team': {}, 'players': {}, 'captain': {}}
    
    # Check if the input is a string
    if isinstance(player_string, str):
        # Split the string by ':'
        parts = player_string.split(':')
        
        # Ensure there are at least two parts
        if len(parts) > 1:
            # Extract the team name
            team_name = parts[0].strip().replace(' Players', '')

            # Extract the player names
            player_names = [name.strip() for name in parts[1].split(',')]

            # Add the team name and player names to the dictionary
            result['team'] = team_name

            # Check for captain names and remove asterisks
            captain_name = next((name.replace('*', '') for name in player_names if '*' in name), None)
            if captain_name:
                result['captain'] = captain_name

            # Remove asterisks from player names, if they exist
            player_names = [player.replace('*', '') for player in player_names]
            result['players'] = player_names

    return result

# Initialize an empty list to store the results for each match
results = []

# Iterate over each row in the DataFrame
for index, row in playing_squad_df.iterrows():
    # Extract players and captain information using the defined function
    match_result = extract_players_and_captain(row['Batter and Bowler Names'])
    
    # Append the results for this match to the list of results
    results.append({'Match ID': row['Match ID'], **match_result})

# Create a DataFrame from the list of results
new_playing_squad_df = pd.DataFrame(results)

# Display the updated DataFrame
new_playing_squad_df.tail()

""


In [7]:
# Initialize empty lists to store full names and short names
full_names = []
short_names = []

# Iterate over each row in the DataFrame
for index, row in new_playing_squad_df.iterrows():
    # Get the list of players for the current team
    players_list = row['players']
    
    # Initialize empty lists to store full names and short names for the current team
    team_full_names = []
    team_short_names = []
    
    # Iterate over each player in the list
    for player in players_list:
        # Split the player's name into first name and last name
        parts = player.split()
        
        # Get the full name
        full_name = ' '.join(parts)
        team_full_names.append(full_name)
        
        # Get the short name
        if len(parts) > 1:
            short_name = f"{parts[0][0]} {parts[-1]}"
        else:
            short_name = player  # Use the full name if only one part exists
        team_short_names.append(short_name)
    
    # Append the full names and short names for the current team to the overall lists
    full_names.append(team_full_names)
    short_names.append(team_short_names)

# Add the 'full name' and 'short name' columns to the DataFrame
new_playing_squad_df['full name'] = full_names
new_playing_squad_df['short name'] = short_names

In [27]:
new_playing_squad_df.to_csv("new_playing_squad_df.csv", index=False)

In [28]:
new_playing_squad_df = pd.read_csv("new_playing_squad_df.csv")
new_playing_squad_df.head()

,Match ID,team,players,captain,full name,short name
0,152,Union Sports Club,"['Abbas Sayed', 'Ali Sumar', 'Aliasgher Jivraj...",{},"['Abbas Sayed', 'Ali Sumar', 'Aliasgher Jivraj...","['A Sayed', 'A Sumar', 'A Jivraj', 'A Kakonzi'..."
1,152,Lions CC,"['Abdulsamad Ahmed', 'Aftab Khamisa', 'Atif Sa...",{},"['Abdulsamad Ahmed', 'Aftab Khamisa', 'Atif Sa...","['A Ahmed', 'A Khamisa', 'A Salim', 'D Parmar'..."
2,153,Alaf Aces,"['Azim Thakur', 'Ejaz Aziz', 'Emmanuel Peter',...",{},"['Azim Thakur', 'Ejaz Aziz', 'Emmanuel Peter',...","['A Thakur', 'E Aziz', 'E Peter', 'H Sharma', ..."
3,153,Lions CC,"['Abdullah Essak', 'Abdulsamad Ahmed', 'Atif S...",{},"['Abdullah Essak', 'Abdulsamad Ahmed', 'Atif S...","['A Essak', 'A Ahmed', 'A Salim', 'D Parmar', ..."
4,157,Annadil Burhani,"['Abbas Adamjee', 'Abdulkadir Dossajee', 'Adna...",{},"['Abbas Adamjee', 'Abdulkadir Dossajee', 'Adna...","['A Adamjee', 'A Dossajee', 'A Zariwala', 'A R..."


### Comes into the Attack Method

In [8]:
# Initialize an empty list to store DataFrames
dfs_to_concat = []

# Iterate through the DataFrame
for index, row in ball_df.iterrows():
    # Check if the value in the 'Batter and Bowler Names' column is not NaN
    if not pd.isna(row['Batter and Bowler Names']):
        # Check if the row contains "comes into the attack" or "comes to the crease"
        if 'comes into the attack' in row['Batter and Bowler Names'] or 'comes to the crease' in row['Batter and Bowler Names']:
            # Extract the full name and match ID from the row
            full_name_parts = row['Batter and Bowler Names'].split(',')[0]
            match_id = row['Match ID']
            # Handle cases with three names
            if len(full_name_parts) == 3:
                first_name_initial = full_name_parts[0][0]
                last_name = full_name_parts[-1]
                full_name = f"{first_name_initial} {last_name}"
            else:
                full_name = full_name_parts
            # Add the full name and match ID to the list of DataFrames
            dfs_to_concat.append(pd.DataFrame({'full name': [full_name], 'Match ID': [match_id]}))

# Concatenate the list of DataFrames into one DataFrame
full_names_df = pd.concat(dfs_to_concat, ignore_index=True)

# Create a new column "short name" by extracting the first letter of the first word and the full second word
full_names_df['short name'] = full_names_df['full name'].apply(lambda x: f"{x.split()[0][0]} {x.split()[-1]}")

# Remove duplicate rows
full_names_df = full_names_df.drop_duplicates()

# Display the DataFrame
print(full_names_df)

                 full name  Match ID    short name
0          Omary Ramadhani      2400   O Ramadhani
1           Ashfaq Imtiyaz      2400     A Imtiyaz
2       Abdulrazak Mohamed      2400     A Mohamed
3             Dylan Manish      2400      D Manish
7        Augustine Mwamele      2400     A Mwamele
12            Laksh Snehal      2400      L Snehal
13          Mohammed Simba      2400       M Simba
14  Sayan Vijay Jobanputra      2400  S Jobanputra
15       Darpan Jobanputra      2400  D Jobanputra
23             Ally Hafidh      2400      A Hafidh
24         Raymond Francis      2400     R Francis
25         Abdullah Jabiri      2400      A Jabiri
28      Zamoyoni Ramadhani      2400   Z Ramadhani
29    Karim Rashidi Kiseto      2400      K Kiseto
34             Umar Shaikh      2400      U Shaikh
35          Mohammed Yunus      2400       M Yunus
43            Halidi Amiri      2400       H Amiri
44            Rehaan Rafiq      2400       R Rafiq
45      John David Maethya     

In [9]:
# Remove "comes to the crease" and any numbers from the "full name" column
full_names_df['full name'] = full_names_df['full name'].str.replace(r' comes to the crease|\d+', '', regex=True).str.strip()

In [56]:
full_names_df.to_csv("cricclubs_players_db.csv", index=False)

In [57]:
full_names_df = pd.read_csv("cricclubs_players_db.csv")
full_names_df.head()

,full name,Match ID,short name
0,Athumani Siwa,196,A Siwa
1,Naran Vekariy,196,N crease
2,Vikram Rathor,196,V Rathor
3,Shravan Kumar,196,S Kumar
4,Ganshyam Gond,196,G crease


### Run and Over Logic

In [10]:
# Replace 'N/A' and blank entries with NaN
ball_by_ball_df['Runs Obtained'].replace(['', 'N/A'], pd.NA, inplace=True)

# Drop rows where 'Runs Obtained' is NaN
ball_by_ball_df = ball_by_ball_df.dropna(subset=['Runs Obtained'])

ball_by_ball_df

,Over Number,Runs Obtained,Batter and Bowler Names,Additional Information,Match ID
6,0.1,0,"A Mohamed to O Ramadhani, 0 run",10:20 AM,2400
7,0.2,0,"A Mohamed to O Ramadhani, 0 run",10:21 AM,2400
8,0.3,1wd,A Mohamed to O Ramadhani WIDE,10:21 AM,2400
9,0.3,4,"A Mohamed to O Ramadhani, 4 runs FOUR",10:21 AM,2400
10,0.4,0,"A Mohamed to O Ramadhani, 0 run",10:21 AM,2400
...,...,...,...,...,...
368,10.2,0,"A Hafidh to B Fredy, 0 run",2:24 PM,2400
369,10.3,0,"A Hafidh to B Fredy, 0 run",2:25 PM,2400
370,10.4,2,"A Hafidh to B Fredy, 2 runs",2:25 PM,2400
371,10.5,0,"A Hafidh to B Fredy, 0 run",2:26 PM,2400


In [11]:
# ball_by_ball_df = df_with_runs

# Convert 'Over Number' column to string type
ball_by_ball_df['Over Number'] = ball_by_ball_df['Over Number'].astype(str)

# Extracting the required information
ball_by_ball_df['over'] = ball_by_ball_df['Over Number'].apply(lambda x: x.split('.')[0])
ball_by_ball_df['delivery'] = ball_by_ball_df['Over Number'].apply(lambda x: x.split('.')[1])

ball_by_ball_df['batter'] = ball_by_ball_df['Batter and Bowler Names'].apply(lambda x: x.split(' to ')[1].split(', ')[0] if ' to ' in x else None)
ball_by_ball_df['bowler'] = ball_by_ball_df['Batter and Bowler Names'].apply(lambda x: x.split(' to ')[0] if ' to ' in x else None)

# Splitting the "Batter and Bowler Names" column after the comma and adding it as "Comments"
ball_by_ball_df['Comments'] = ball_by_ball_df['Batter and Bowler Names']
# Reorder the columns with the new "Comments" column
ball_by_ball_df = ball_by_ball_df[['Match ID', 'over', 'delivery', 'batter', 'bowler', 'Comments', 'Runs Obtained', 'Additional Information']]

# Starting overs from 1 instead of 0
overs = []
for over in ball_by_ball_df['over']:
    over2 = int(over) + 1
    overs.append(over2)

ball_by_ball_df['over'] = overs

# Print the resulting DataFrame
print(ball_by_ball_df.head())

    Match ID  over delivery            batter     bowler  \
6       2400     1        1       O Ramadhani  A Mohamed   
7       2400     1        2       O Ramadhani  A Mohamed   
8       2400     1        3  O Ramadhani WIDE  A Mohamed   
9       2400     1        3       O Ramadhani  A Mohamed   
10      2400     1        4       O Ramadhani  A Mohamed   

                                  Comments Runs Obtained  \
6          A Mohamed to O Ramadhani, 0 run             0   
7          A Mohamed to O Ramadhani, 0 run             0   
8            A Mohamed to O Ramadhani WIDE           1wd   
9   A Mohamed to O Ramadhani, 4 runs  FOUR             4   
10         A Mohamed to O Ramadhani, 0 run             0   

   Additional Information  
6                10:20 AM  
7                10:21 AM  
8                10:21 AM  
9                10:21 AM  
10               10:21 AM  


C:\Users\User\AppData\Local\Temp\ipykernel_16740\3552655546.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ball_by_ball_df['over'] = overs


In [12]:
# Get unique match_ids from the DataFrame
unique_match_ids = ball_by_ball_df['Match ID'].unique()

# Initialize a variable to keep track of the inning number
inning_number = 1

# Initialize an empty list to store inning numbers
inning_numbers = []

# Iterate over the unique match_ids
for match_id in unique_match_ids:
    inning_number = 1

    # Filter the DataFrame for the current match_id
    match_df = ball_by_ball_df[ball_by_ball_df['Match ID'] == match_id]
    
    # Initialize a variable to keep track of the previous over number
    prev_over = None
    
    # Iterate over the rows of the filtered DataFrame
    for index, row in match_df.iterrows():
        # Check if the current over number is less than the previous over number
        if prev_over is not None and row['over'] < prev_over:
            # Increment the inning number if the current over is less than the previous over
            inning_number += 1
        
        # Append the current inning number to the list
        inning_numbers.append(inning_number)
        
        # Update the previous over number for the next iteration
        prev_over = row['over']

# Add the inning numbers to the DataFrame
ball_by_ball_df['inning_number'] = inning_numbers

# Print the DataFrame with the added inning numbers
print(ball_by_ball_df.head())

    Match ID  over delivery            batter     bowler  \
6       2400     1        1       O Ramadhani  A Mohamed   
7       2400     1        2       O Ramadhani  A Mohamed   
8       2400     1        3  O Ramadhani WIDE  A Mohamed   
9       2400     1        3       O Ramadhani  A Mohamed   
10      2400     1        4       O Ramadhani  A Mohamed   

                                  Comments Runs Obtained  \
6          A Mohamed to O Ramadhani, 0 run             0   
7          A Mohamed to O Ramadhani, 0 run             0   
8            A Mohamed to O Ramadhani WIDE           1wd   
9   A Mohamed to O Ramadhani, 4 runs  FOUR             4   
10         A Mohamed to O Ramadhani, 0 run             0   

   Additional Information  inning_number  
6                10:20 AM              1  
7                10:21 AM              1  
8                10:21 AM              1  
9                10:21 AM              1  
10               10:21 AM              1  


C:\Users\User\AppData\Local\Temp\ipykernel_16740\2083204241.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ball_by_ball_df['inning_number'] = inning_numbers


In [13]:
ball_by_ball_df.describe()

,Match ID,over,inning_number
count,424.0,424.000000,424.000000
mean,2385.0,17.313679,1.483491
std,0.0,9.838565,0.500318
min,2385.0,1.000000,1.000000
25%,2385.0,9.000000,1.000000
50%,2385.0,17.000000,1.000000
75%,2385.0,26.000000,2.000000
max,2385.0,35.000000,2.000000


In [33]:
ball_by_ball_df.to_csv("ball_by_ball_df_initial.csv", index=False)

In [34]:
ball_by_ball_df = pd.read_csv("ball_by_ball_df_initial.csv")
ball_by_ball_df.head()

,Match ID,over,delivery,batter,bowler,Comments,Runs Obtained,Additional Information,inning_number
0,2,1,1,S Halari,A Nagewadia,"A Nagewadia to S Halari, 0 run",0,4:20 PM,1
1,2,1,2,S Halari,A Nagewadia,"A Nagewadia to S Halari, 0 run",0,4:20 PM,1
2,2,1,3,S Halari,A Nagewadia,"A Nagewadia to S Halari, 0 run",0,4:20 PM,1
3,2,1,4,S Halari,A Nagewadia,"A Nagewadia to S Halari, 0 run",0,4:20 PM,1
4,2,1,5,S Halari,A Nagewadia,"A Nagewadia to S Halari, 2 runs",2,4:20 PM,1


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_team_innings_mapping(match_id):
    url = f'https://www.cricclubs.com/Tanzania/ballbyball.do?matchId={match_id}&clubId=7605'
    
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')

    batting_team_elements = soup.find_all('li', id=lambda x: x and x.startswith('ballByBallTeamTab'))
    batting_teams = [element.find('a').text.strip() for element in batting_team_elements]
    inning_numbers = [i + 1 for i in range(len(batting_teams))]
    bowling_team = batting_teams[::-1]

    # Check the length of the batting_teams list
    if len(batting_teams) > 1:
        # If the length is greater than 1, create inning_numbers as usual
        bowling_team = batting_teams[::-1] # Reverse the order for the second inning
    else:
        # Find all 'li' elements with the class 'win'
        team_elements = soup.find_all('li', class_='win')

        # Extract the team names from the 'span' elements with class 'teamName'
        team_names = [element.find('span', class_='teamName').text.strip() for element in team_elements]
        
        # Check if the teams in team_names are present in batting_teams
        for team in team_names:
            if team not in batting_teams:
                bowling_team = team
    
    # Create a DataFrame for the team innings mapping
    team_inning_mapping = pd.DataFrame({
        'Match ID': match_id, 
        'Inning Number': inning_numbers,
        'Batting Team': batting_teams,
        'Bowling Team': bowling_team
    })
    
    return team_inning_mapping


In [16]:
team_innings_mapping_df = get_team_innings_mapping(2400)
team_innings_mapping_df

,Match ID,Inning Number,Batting Team,Bowling Team
0,2400,1,Kilimanjaro Boys,Serengeti Boys
1,2400,2,Serengeti Boys,Kilimanjaro Boys


In [ ]:
# Initialize an empty list to store all scraped data
all_data = []

# Initialize empty lists to store successful and failed match IDs
success_match_ids = []
failed_match_ids = []

# Initialize fail count
fail_count = 0

# Loop through the match IDs
for i in range(1, 2300):
    # Convert the match ID to the specified format
    if i < 10:
        match_id = f'0{i}'
    elif 10 <= i < 100:
        match_id = f'{i}'
    elif 100 <= i < 1000:
        match_id = f'{i}'
    else:
        match_id = f'{i}'

    # Scrape data for the current match ID
    match_data = get_team_innings_mapping(match_id)

    # Check if the match data is None (indicating a failed request)
    if match_data is None:
        # Increment the fail counter
        fail_count += 1
        # Append the failed match ID to the list of failed match IDs
        failed_match_ids.append(match_id)
        # If the fail count is more than 5, stop the loop
        if fail_count > 5:
            break
    else:
        # Reset the fail counter
        fail_count = 0
        # Append the successful match ID to the list of successful match IDs
        success_match_ids.append(match_id)
        # Append the match data to the list of all data
        all_data.append(match_data)

# Concatenate all scraped data into one DataFrame
team_innings_mapping_df = pd.concat(all_data, ignore_index=True)

team_innings_mapping_df


In [166]:
team_innings_mapping_df.to_csv("batting_and_bowling_teams.csv", index=False)

In [36]:
team_innings_mapping_df = pd.read_csv("batting_and_bowling_teams.csv")
team_innings_mapping_df.head()

,Match ID,Inning Number,Batting Team,Bowling Team
0,2,1.0,Aksc,Gymkhana
1,2,2.0,Gymkhana,Aksc
2,3,1.0,Lions CC,Sklpsc
3,3,2.0,Sklpsc,Lions CC
4,4,1.0,Gymkhana,Estim


In [17]:
team_innings_mapping_df['Inning Number'] = team_innings_mapping_df['Inning Number'].astype(int)

team_innings_mapping_df.rename(columns={'Inning Number': 'inning_number', 'Batting Team': 'batting_team', 'Bowling Team': 'bowling_team'}, inplace=True)

In [18]:
# Merge the two dataframes based on 'Match ID' and 'inning_number' using a left join
ball_by_ball_df = ball_by_ball_df.merge(team_innings_mapping_df, on=['Match ID', 'inning_number'], how='left')

ball_by_ball_df.head()

,Match ID,over,delivery,batter,bowler,Comments,Runs Obtained,Additional Information,inning_number,batting_team,bowling_team
0,2400,1,1,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 0 run",0,10:20 AM,1,Kilimanjaro Boys,Serengeti Boys
1,2400,1,2,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 0 run",0,10:21 AM,1,Kilimanjaro Boys,Serengeti Boys
2,2400,1,3,O Ramadhani WIDE,A Mohamed,A Mohamed to O Ramadhani WIDE,1wd,10:21 AM,1,Kilimanjaro Boys,Serengeti Boys
3,2400,1,3,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 4 runs FOUR",4,10:21 AM,1,Kilimanjaro Boys,Serengeti Boys
4,2400,1,4,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 0 run",0,10:21 AM,1,Kilimanjaro Boys,Serengeti Boys


In [19]:
# 1. Remove the words 'WIDE' and 'NO BALL'
ball_by_ball_df['batter'] = ball_by_ball_df['batter'].str.replace('WIDES', '').str.replace('WIDE', '').str.replace('NO BALL', '')

# 2. Remove the word 'OUT' and everything after it

# Define a regex pattern to match everything before 'OUT' if 'OUT' exists
pattern = r'^(.*?)(OUT|$)'

# Apply regex pattern to the 'text_column' using str.extract()
ball_by_ball_df['batter'] = ball_by_ball_df['batter'].str.extract(pattern, expand=False)[0].str.strip()

# 3. Remove any numbers
ball_by_ball_df['batter'] = ball_by_ball_df['batter'].str.replace('\d+', '')

# Trim extra whitespace
ball_by_ball_df['batter'] = ball_by_ball_df['batter'].str.strip()

# Display the cleaned DataFrame
ball_by_ball_df.head()

,Match ID,over,delivery,batter,bowler,Comments,Runs Obtained,Additional Information,inning_number,batting_team,bowling_team
0,2400,1,1,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 0 run",0,10:20 AM,1,Kilimanjaro Boys,Serengeti Boys
1,2400,1,2,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 0 run",0,10:21 AM,1,Kilimanjaro Boys,Serengeti Boys
2,2400,1,3,O Ramadhani,A Mohamed,A Mohamed to O Ramadhani WIDE,1wd,10:21 AM,1,Kilimanjaro Boys,Serengeti Boys
3,2400,1,3,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 4 runs FOUR",4,10:21 AM,1,Kilimanjaro Boys,Serengeti Boys
4,2400,1,4,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 0 run",0,10:21 AM,1,Kilimanjaro Boys,Serengeti Boys


In [20]:
# Iterate over the rows to process the 'RETIRED' cases
for i in range(len(ball_by_ball_df)):
    if 'RETIRED' in ball_by_ball_df.loc[i, 'Comments']:
        comment = ball_by_ball_df.loc[i, 'Comments']
        delivery_no = ball_by_ball_df.loc[i, 'delivery']
        
        # Extract batter's name
        batter_name = comment.split(',')[0].split('\n')[0].strip()
        
        if int(delivery_no) > 1:
            # Append over and bowler name of the previous ball
            ball_by_ball_df.loc[i, 'bowler'] = ball_by_ball_df.loc[i - 1, 'bowler']
        else:
            # Append over and bowler name of the next ball
            ball_by_ball_df.loc[i, 'bowler'] = ball_by_ball_df.loc[i + 1, 'bowler']
        
        # Append the batter name that appears at the start of the comment
        ball_by_ball_df.loc[i, 'batter'] = batter_name

# Display the updated DataFrame
print(ball_by_ball_df)

     Match ID  over delivery       batter     bowler  \
0        2400     1        1  O Ramadhani  A Mohamed   
1        2400     1        2  O Ramadhani  A Mohamed   
2        2400     1        3  O Ramadhani  A Mohamed   
3        2400     1        3  O Ramadhani  A Mohamed   
4        2400     1        4  O Ramadhani  A Mohamed   
..        ...   ...      ...          ...        ...   
294      2400    11        2      B Fredy   A Hafidh   
295      2400    11        3      B Fredy   A Hafidh   
296      2400    11        4      B Fredy   A Hafidh   
297      2400    11        5      B Fredy   A Hafidh   
298      2400    11        6      B Fredy   A Hafidh   

                                   Comments Runs Obtained  \
0           A Mohamed to O Ramadhani, 0 run             0   
1           A Mohamed to O Ramadhani, 0 run             0   
2             A Mohamed to O Ramadhani WIDE           1wd   
3    A Mohamed to O Ramadhani, 4 runs  FOUR             4   
4           A Mohamed 

In [21]:
# Isolate rows with "WIDE", "WIDES", and "OUT"
retired_rows = ball_by_ball_df[ball_by_ball_df['Comments'].str.contains('RETIRED')]
wide_rows = ball_by_ball_df[ball_by_ball_df['Comments'].str.contains('WIDE|WIDES')]
out_rows = ball_by_ball_df[ball_by_ball_df['Comments'].str.contains('OUT')]

# Perform split function on the rest of the rows
other_rows = ball_by_ball_df[~ball_by_ball_df['Comments'].str.contains('WIDE|WIDES|OUT')]
other_rows[['ball_outcome', 'ball_details']] = other_rows['Comments'].str.split(',', n=1, expand=True)

# Isolate numbers for WIDE and WIDES columns
wide_rows[['ball_outcome', 'ball_details']] = wide_rows['Comments'].str.extract(r'(\d*)\s*(WIDE|WIDES)')

# Delete everything before OUT for OUT rows
out_rows['ball_outcome'] = 'OUT'
out_rows['ball_details'] = out_rows['Comments'].str.split('OUT!').str[1]

print(wide_rows.head())
print(out_rows.head())

    Match ID  over delivery       batter     bowler  \
2       2400     1        3  O Ramadhani  A Mohamed   
11      2400     2        5  O Ramadhani   D Manish   
26      2400     5        1  O Ramadhani  A Mohamed   
28      2400     5        2    A Imtiyaz  A Mohamed   
29      2400     5        2    A Imtiyaz  A Mohamed   

                         Comments Runs Obtained Additional Information  \
2   A Mohamed to O Ramadhani WIDE           1wd               10:21 AM   
11   D Manish to O Ramadhani WIDE           1wd               10:21 AM   
26  A Mohamed to O Ramadhani WIDE           1wd               10:22 AM   
28    A Mohamed to A Imtiyaz WIDE           1wd               10:22 AM   
29    A Mohamed to A Imtiyaz WIDE           1wd               10:22 AM   

    inning_number      batting_team    bowling_team ball_outcome ball_details  
2               1  Kilimanjaro Boys  Serengeti Boys                      WIDE  
11              1  Kilimanjaro Boys  Serengeti Boys             

C:\Users\User\AppData\Local\Temp\ipykernel_16740\600072775.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_rows[['ball_outcome', 'ball_details']] = other_rows['Comments'].str.split(',', n=1, expand=True)
C:\Users\User\AppData\Local\Temp\ipykernel_16740\600072775.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_rows[['ball_outcome', 'ball_details']] = other_rows['Comments'].str.split(',', n=1, expand=True)
C:\Users\User\AppData\Local\Temp\ipykernel_16740\600072775.py:11: SettingWithCopyWar

In [24]:
retired_rows.head()

,Match ID,over,delivery,batter,bowler,Comments,Runs Obtained,Additional Information,inning_number,batting_team,bowling_team


In [43]:
other_rows.head()

,Match ID,over,delivery,batter,bowler,Comments,Runs Obtained,Additional Information,inning_number,batting_team,bowling_team,ball_outcome,ball_details
0,2,1,1,S Halari,A Nagewadia,"A Nagewadia to S Halari, 0 run",0,4:20 PM,1,Aksc,Gymkhana,A Nagewadia to S Halari,0 run
1,2,1,2,S Halari,A Nagewadia,"A Nagewadia to S Halari, 0 run",0,4:20 PM,1,Aksc,Gymkhana,A Nagewadia to S Halari,0 run
2,2,1,3,S Halari,A Nagewadia,"A Nagewadia to S Halari, 0 run",0,4:20 PM,1,Aksc,Gymkhana,A Nagewadia to S Halari,0 run
3,2,1,4,S Halari,A Nagewadia,"A Nagewadia to S Halari, 0 run",0,4:20 PM,1,Aksc,Gymkhana,A Nagewadia to S Halari,0 run
4,2,1,5,S Halari,A Nagewadia,"A Nagewadia to S Halari, 2 runs",2,4:20 PM,1,Aksc,Gymkhana,A Nagewadia to S Halari,2 runs


In [22]:
import re
import pandas as pd

# Define a function to clean the ball_details column
def clean_ball_details(comment):
    return comment.replace('\n', '').replace('\r', '').replace('\xa0', ' ')

# Clean the ball_details column
out_rows['ball_details'] = out_rows['ball_details'].apply(clean_ball_details)

# Define a function to isolate specified words
def isolate_word(comment):
    words_to_isolate = ['CAUGHT & BOWLED', 'LBW', 'CAUGHT', 'BOWLED', 'STUMPED', 'RUN OUT', 'HIT WICKET', 'OBSTRUCTING THE FIELD']
    for word in words_to_isolate:
        if word in comment:
            return word.lower()

# Apply the function to isolate words in the cleaned "ball_details" column
out_rows['wicket_type'] = out_rows['ball_details'].apply(isolate_word)

# Extract the full names and short names from full_names_df
player_full_names = full_names_df['full name'].tolist()
player_short_names = full_names_df['short name'].tolist()

# Define a function to check if any player names are present in the cleaned ball_details column
def check_player_names(comment):
    for player_name in player_full_names + player_short_names:
        if player_name in comment:
            return player_name
    return None

# Apply the function to check player names in the cleaned ball_details column
out_rows['player_out'] = out_rows['ball_details'].apply(check_player_names)

# Check if the "wicket_type" column contains two words
def join_words(text):
    if text is not None:
        words = text.split()
        if len(words) == 2:
            return '_'.join(words)
        return text
    return ''

# Apply the function to join the words in the "wicket_type" column
out_rows['wicket_type'] = out_rows['wicket_type'].apply(join_words)

# Define a function to extract runs scored during run outs and extras
def extract_runs(comment):
    # Use regex to search for the pattern 'X run' where X is a number
    match = re.search(r'(\d+) run', comment)
    if match:
        return int(match.group(1))
    return 0

# Apply the function to extract runs scored during run outs for all rows
out_rows['total_runs'] = out_rows['Comments'].apply(extract_runs)

# Define a function to handle extras
def extract_extras(comment, extra_type):
    if extra_type in comment:
        match = re.search(r'(\d+)', comment)
        if match:
            return int(match.group(1))
    return 0

# Define a function to update batter_runs and extras_runs
def update_runs(row):
    if 'RUN OUT' in row['ball_details']:
        row['batter_runs'] = row['total_runs'] - row['extras_runs']
    else:
        row['batter_runs'] = row['total_runs']
    return row

# Initialize columns for extras
out_rows['extras_runs'] = 0
out_rows['extras_type'] = None

# Process each type of extra
extra_types = ['LEG BYE', 'BYE', 'NO BALL', 'WIDE']
for extra_type in extra_types:
    out_rows['extras_runs'] = out_rows.apply(lambda row: extract_extras(row['ball_details'], extra_type) if extra_type in row['ball_details'] else row['extras_runs'], axis=1)
    out_rows['extras_type'] = out_rows.apply(lambda row: extra_type.lower().replace(' ', '_') if extra_type in row['ball_details'] else row['extras_type'], axis=1)

# Update batter_runs based on total_runs and extras_runs
out_rows = out_rows.apply(update_runs, axis=1)

# Define a function to determine the bowler's wicket
def get_bowler_wicket(wicket_type):
    if wicket_type in ['run_out', 'obstructing_the_field']:
        return 0
    else:
        return 1

# Apply the function to create the new column
out_rows['bowler_wicket'] = out_rows['wicket_type'].apply(lambda x: get_bowler_wicket(x))

# Set the wickets column to 1 for all rows
out_rows['wickets'] = 1

# Display the updated DataFrame
print(out_rows.head())

     Match ID  over delivery        batter        bowler  \
35       2400     5        6     A Imtiyaz     A Mohamed   
67       2400    10        6     A Mwamele      L Snehal   
70       2400    11        3   O Ramadhani  S Jobanputra   
117      2400    18        5       M Simba      L Snehal   
121      2400    19        3  D Jobanputra     R Francis   

                                              Comments Runs Obtained  \
35   A Mohamed to A Imtiyaz OUT!\nBOWLED  \n\nAshfa...             W   
67   L Snehal to A Mwamele OUT!\nCAUGHT \n\nAugusti...             W   
70   S Jobanputra to O Ramadhani OUT!\nCAUGHT \n\nO...             W   
117  L Snehal to M Simba OUT!\nBOWLED  \n\nMohammed...             W   
121  R Francis to D Jobanputra OUT!\nCAUGHT \n\nDar...             W   

    Additional Information  inning_number      batting_team  ... ball_outcome  \
35                10:24 AM              1  Kilimanjaro Boys  ...          OUT   
67                10:44 AM              1  K

C:\Users\User\AppData\Local\Temp\ipykernel_16740\3884270045.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_rows['ball_details'] = out_rows['ball_details'].apply(clean_ball_details)
C:\Users\User\AppData\Local\Temp\ipykernel_16740\3884270045.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_rows['wicket_type'] = out_rows['ball_details'].apply(isolate_word)
C:\Users\User\AppData\Local\Temp\ipykernel_16740\3884270045.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [23]:
out_rows.head()

,Match ID,over,delivery,batter,bowler,Comments,Runs Obtained,Additional Information,inning_number,batting_team,...,ball_outcome,ball_details,wicket_type,player_out,total_runs,extras_runs,extras_type,batter_runs,bowler_wicket,wickets
35,2400,5,6,A Imtiyaz,A Mohamed,A Mohamed to A Imtiyaz OUT!\nBOWLED \n\nAshfa...,W,10:24 AM,1,Kilimanjaro Boys,...,OUT,BOWLED Ashfaq Imtiyaz ...,bowled,Ashfaq Imtiyaz,0,0,None,0,1,1
67,2400,10,6,A Mwamele,L Snehal,L Snehal to A Mwamele OUT!\nCAUGHT \n\nAugusti...,W,10:44 AM,1,Kilimanjaro Boys,...,OUT,CAUGHT Augustine Mwamele ...,caught,Augustine Mwamele,0,0,None,0,1,1
70,2400,11,3,O Ramadhani,S Jobanputra,S Jobanputra to O Ramadhani OUT!\nCAUGHT \n\nO...,W,10:47 AM,1,Kilimanjaro Boys,...,OUT,CAUGHT Omary Ramadhani ...,caught,Omary Ramadhani,0,0,None,0,1,1
117,2400,18,5,M Simba,L Snehal,L Snehal to M Simba OUT!\nBOWLED \n\nMohammed...,W,11:19 AM,1,Kilimanjaro Boys,...,OUT,BOWLED Mohammed Simba ...,bowled,Mohammed Simba,0,0,None,0,1,1
121,2400,19,3,D Jobanputra,R Francis,R Francis to D Jobanputra OUT!\nCAUGHT \n\nDar...,W,11:30 AM,1,Kilimanjaro Boys,...,OUT,CAUGHT Darpan Jobanputra ...,caught,Darpan Jobanputra,0,0,None,0,1,1


In [24]:
out_rows.iloc[207]

IndexError: single positional indexer is out-of-bounds

In [25]:
# 1. If ball_outcome is empty (blank), append 1 to a new column called "extras_runs". 
# If not empty, append whatever is in the ball_outcome column to the extras_runs column.
wide_rows['extras_runs'] = wide_rows['ball_outcome'].apply(lambda x: 1 if x.strip() == '' else x)

# 2. Append ball_details.lower() to a new column called extras_type
wide_rows['extras_type'] = wide_rows['ball_details'].str.lower() + 's'

wide_rows['total_runs'] = wide_rows['extras_runs']
wide_rows['batter_runs'] = 0
wide_rows['wickets'] = 0
wide_rows['wicket_type'] = None
wide_rows['player_out'] = None
wide_rows['bowler_wicket'] = 0

# Display the updated DataFrame
wide_rows.head()

C:\Users\User\AppData\Local\Temp\ipykernel_16740\2033951784.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wide_rows['extras_runs'] = wide_rows['ball_outcome'].apply(lambda x: 1 if x.strip() == '' else x)
C:\Users\User\AppData\Local\Temp\ipykernel_16740\2033951784.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wide_rows['extras_type'] = wide_rows['ball_details'].str.lower() + 's'
C:\Users\User\AppData\Local\Temp\ipykernel_16740\2033951784.py:8: SettingWithCopyWarning: 
A value is trying to be set 

,Match ID,over,delivery,batter,bowler,Comments,Runs Obtained,Additional Information,inning_number,batting_team,...,ball_outcome,ball_details,extras_runs,extras_type,total_runs,batter_runs,wickets,wicket_type,player_out,bowler_wicket
2,2400,1,3,O Ramadhani,A Mohamed,A Mohamed to O Ramadhani WIDE,1wd,10:21 AM,1,Kilimanjaro Boys,...,,WIDE,1,wides,1,0,0,None,None,0
11,2400,2,5,O Ramadhani,D Manish,D Manish to O Ramadhani WIDE,1wd,10:21 AM,1,Kilimanjaro Boys,...,,WIDE,1,wides,1,0,0,None,None,0
26,2400,5,1,O Ramadhani,A Mohamed,A Mohamed to O Ramadhani WIDE,1wd,10:22 AM,1,Kilimanjaro Boys,...,,WIDE,1,wides,1,0,0,None,None,0
28,2400,5,2,A Imtiyaz,A Mohamed,A Mohamed to A Imtiyaz WIDE,1wd,10:22 AM,1,Kilimanjaro Boys,...,,WIDE,1,wides,1,0,0,None,None,0
29,2400,5,2,A Imtiyaz,A Mohamed,A Mohamed to A Imtiyaz WIDE,1wd,10:22 AM,1,Kilimanjaro Boys,...,,WIDE,1,wides,1,0,0,None,None,0


In [62]:
specific_row = other_rows.iloc[157077]
specific_row

Match ID                                                  924
over                                                       16
delivery                                                    5
batter                                                F Salum
bowler                                                A Mpeka
Comments                  A Mpeka to F Salum, 7 runs  PENALTY
Runs Obtained                                              7P
Additional Information                               11:16 AM
inning_number                                               1
batting_team                                     Kingalu Boys
bowling_team                                   Morogoro Stars
ball_outcome                               A Mpeka to F Salum
ball_details                                  7 runs  PENALTY
Name: 181193, dtype: object

In [26]:
# Define a function to extract numbers from a string
def extract_number(text):
    words = text.split()
    for word in words:
        if word.isdigit():
            return int(word)
    return None

# 1. If ball_details contains "LEG BYE", append the number in the column to a new column called "extras_runs"
# and "legbyes" to a new column called "extras_type"
other_rows['extras_runs'] = other_rows['ball_details'].apply(lambda x: extract_number(x) if x is not None and 'LEG BYE' in x else None)
other_rows['extras_type'] = other_rows['ball_details'].apply(lambda x: 'legbyes' if x is not None and 'LEG BYE' in x else None)

# 2. If ball_details contains "BYE", append the number in the column to the column "extras_runs" 
# and "byes" to the column "extras_type"
other_rows['extras_runs'].fillna(other_rows['ball_details'].apply(lambda x: extract_number(x) if x is not None and 'BYE' in x else None), inplace=True)
other_rows['extras_type'].fillna(other_rows['ball_details'].apply(lambda x: 'byes' if x is not None and 'BYE' in x else None), inplace=True)

# 3. If ball_details contains "NO BALL", append the number in the column to a new column called "extras_runs"
# and "noballs" to a new column called "extras_type"
other_rows['extras_runs'].fillna(other_rows['ball_details'].apply(lambda x: 1 if x is not None and 'NO BALL' in x else None), inplace=True)
other_rows['extras_type'].fillna(other_rows['ball_details'].apply(lambda x: 'noballs' if x is not None and 'NO BALL' in x else None), inplace=True)

# 4. If neither "LEG BYE" nor "BYE" exists in ball_details, append the numbers to a new column called "total_runs"
other_rows['total_runs'] = other_rows['ball_details'].apply(lambda x: extract_number(x) if x is not None and extract_number(x) is not None else None)

# Define a function to determine the value of batter_runs based on the conditions
def calculate_batter_runs(row):
    if row['extras_type'] == 'legbyes' or row['extras_type'] == 'byes':
        return 0
    elif row['extras_type'] == 'noballs':
        return row['total_runs'] - 1
    else:
        return row['total_runs']

# Add a column called "batter_runs" based on the conditions
other_rows['batter_runs'] = other_rows.apply(calculate_batter_runs, axis=1)

other_rows['wickets'] = 0
other_rows['wicket_type'] = None
other_rows['player_out'] = None
other_rows['bowler_wicket'] = 0

for index, row in other_rows.iterrows():
    if 'RETIRED' in row['Comments']:
        comment = row['Comments']  # Update the comment variable
        retired_name = comment.split('\n')[-1].split(' is RETIRED')[0].strip()
        other_rows.loc[index, 'player_out'] = retired_name
        other_rows.loc[index, 'wicket_type'] = 'retired_hurt'
        
        # Check if 'total_runs' is null and fill it with 0 if it is
        if pd.isnull(row['total_runs']):
            other_rows.loc[index, 'total_runs'] = 0
        if pd.isnull(row['batter_runs']):
            other_rows.loc[index, 'batter_runs'] = 0
        if pd.isnull(row['extras_runs']):
            other_rows.loc[index, 'extras_runs'] = 0
        if pd.isnull(row['extras_type']):
            other_rows.loc[index, 'extras_type'] = None
            
    if 'PENALTY' in row['Comments']:
        other_rows.loc[index, 'batter_runs'] = 0
        other_rows.loc[index, 'extras_type'] = 'penalty_runs'

# Display the updated DataFrame
other_rows.head()

C:\Users\User\AppData\Local\Temp\ipykernel_16740\1543781403.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_rows['extras_runs'] = other_rows['ball_details'].apply(lambda x: extract_number(x) if x is not None and 'LEG BYE' in x else None)
C:\Users\User\AppData\Local\Temp\ipykernel_16740\1543781403.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_rows['extras_type'] = other_rows['ball_details'].apply(lambda x: 'legbyes' if x is not None and 'LEG BYE' in x else None)
C:\Users\User\AppData\

,Match ID,over,delivery,batter,bowler,Comments,Runs Obtained,Additional Information,inning_number,batting_team,...,ball_outcome,ball_details,extras_runs,extras_type,total_runs,batter_runs,wickets,wicket_type,player_out,bowler_wicket
0,2400,1,1,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 0 run",0,10:20 AM,1,Kilimanjaro Boys,...,A Mohamed to O Ramadhani,0 run,NaN,None,0,0,0,None,None,0
1,2400,1,2,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 0 run",0,10:21 AM,1,Kilimanjaro Boys,...,A Mohamed to O Ramadhani,0 run,NaN,None,0,0,0,None,None,0
3,2400,1,3,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 4 runs FOUR",4,10:21 AM,1,Kilimanjaro Boys,...,A Mohamed to O Ramadhani,4 runs FOUR,NaN,None,4,4,0,None,None,0
4,2400,1,4,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 0 run",0,10:21 AM,1,Kilimanjaro Boys,...,A Mohamed to O Ramadhani,0 run,NaN,None,0,0,0,None,None,0
5,2400,1,5,O Ramadhani,A Mohamed,"A Mohamed to O Ramadhani, 0 run",0,10:21 AM,1,Kilimanjaro Boys,...,A Mohamed to O Ramadhani,0 run,NaN,None,0,0,0,None,None,0


In [27]:
# Concatenate the dataframes
merged_df = pd.concat([out_rows, wide_rows, other_rows], ignore_index=True)

# Rearrange the columns
merged_df = merged_df[['Match ID', 'inning_number', 'batting_team', 'bowling_team', 'over', 'delivery', 'batter', 'bowler', 'total_runs', 'batter_runs', 
                       'extras_runs', 'extras_type', 'wickets', 'bowler_wicket', 'player_out', 
                       'wicket_type']]

# Sort the DataFrame by multiple columns in the specified order
merged_df = merged_df.sort_values(by=['Match ID', 'inning_number', 'over', 'delivery'])

# Remove any numbers from the "batter" column
merged_df['batter'] = merged_df['batter'].str.replace(r'\d+', '', regex=True).str.strip().str.replace(',', '')

# Display the merged dataframe
merged_df.head()

,Match ID,inning_number,batting_team,bowling_team,over,delivery,batter,bowler,total_runs,batter_runs,extras_runs,extras_type,wickets,bowler_wicket,player_out,wicket_type
29,2400,1,Kilimanjaro Boys,Serengeti Boys,1,1,O Ramadhani,A Mohamed,0,0,NaN,None,0,0,None,None
30,2400,1,Kilimanjaro Boys,Serengeti Boys,1,2,O Ramadhani,A Mohamed,0,0,NaN,None,0,0,None,None
12,2400,1,Kilimanjaro Boys,Serengeti Boys,1,3,O Ramadhani,A Mohamed,1,0,1.0,wides,0,0,None,None
31,2400,1,Kilimanjaro Boys,Serengeti Boys,1,3,O Ramadhani,A Mohamed,4,4,NaN,None,0,0,None,None
32,2400,1,Kilimanjaro Boys,Serengeti Boys,1,4,O Ramadhani,A Mohamed,0,0,NaN,None,0,0,None,None


In [28]:
merged_df.to_csv("merged_df.csv", index=False)

In [67]:
merged_df = pd.read_csv("merged_df.csv")
merged_df.tail()

,Match ID,inning_number,batting_team,bowling_team,over,delivery,batter,bowler,total_runs,batter_runs,extras_runs,extras_type,wickets,bowler_wicket,player_out,wicket_type
479540,2214,2,Mrisho Primary Girls,Baptist Girls,14,2,E Kusekwa,R Sheko,2.0,2.0,NaN,NaN,0,0,NaN,NaN
479541,2214,2,Mrisho Primary Girls,Baptist Girls,14,3,E Kusekwa,R Sheko,0.0,0.0,NaN,NaN,0,0,NaN,NaN
479542,2214,2,Mrisho Primary Girls,Baptist Girls,14,4,E Kusekwa,R Sheko,0.0,0.0,NaN,NaN,0,0,NaN,NaN
479543,2214,2,Mrisho Primary Girls,Baptist Girls,14,5,E Kusekwa,R Sheko,0.0,0.0,NaN,NaN,0,0,NaN,NaN
479544,2214,2,Mrisho Primary Girls,Baptist Girls,14,6,E Kusekwa,R Sheko,0.0,0.0,0.0,NaN,1,1,Ester Kusekwa,caught


### Replacing Short Names with Full - Team List Method

In [29]:
new_playing_squad_df.tail()
merged_df.tail()

,Match ID,inning_number,batting_team,bowling_team,over,delivery,batter,bowler,total_runs,batter_runs,extras_runs,extras_type,wickets,bowler_wicket,player_out,wicket_type
294,2400,2,Serengeti Boys,Kilimanjaro Boys,11,2,B Fredy,A Hafidh,0,0,NaN,None,0,0,None,None
295,2400,2,Serengeti Boys,Kilimanjaro Boys,11,3,B Fredy,A Hafidh,0,0,NaN,None,0,0,None,None
296,2400,2,Serengeti Boys,Kilimanjaro Boys,11,4,B Fredy,A Hafidh,2,2,NaN,None,0,0,None,None
297,2400,2,Serengeti Boys,Kilimanjaro Boys,11,5,B Fredy,A Hafidh,0,0,NaN,None,0,0,None,None
298,2400,2,Serengeti Boys,Kilimanjaro Boys,11,6,B Fredy,A Hafidh,0,0,NaN,None,0,0,None,None


In [31]:
new_playing_squad_df.tail()

,full name,short name


In [30]:
# Iterate over each row in the merged_df
for index, row in merged_df.iterrows():
    # Extract match ID, batting team, batter, and bowler information from the current row
    match_id = row['Match ID']
    batting_team = row['batting_team']
    bowling_team = row['bowling_team']
    batter_short_name = row['batter']
    bowler_short_name = row['bowler']
    
    # Find the corresponding rows in new_playing_squad_df based on match ID and team
    bat_squad_rows = new_playing_squad_df[(new_playing_squad_df['Match ID'] == match_id) & (new_playing_squad_df['team'] == batting_team)]
    bowl_squad_rows = new_playing_squad_df[(new_playing_squad_df['Match ID'] == match_id) & (new_playing_squad_df['team'] == bowling_team)]
    
    # Check if there are matching rows for batting and bowling teams
    if not bat_squad_rows.empty and not bowl_squad_rows.empty:
        # Find the full name of the batter and bowler using their short names
        bat_squad_row = bat_squad_rows.iloc[0]
        bowl_squad_row = bowl_squad_rows.iloc[0]
        
        # Check if the short name exists in the list before retrieving its index
        if batter_short_name in bat_squad_row['short name']:
            batter_full_name = bat_squad_row['full name'][bat_squad_row['short name'].index(batter_short_name)]
            merged_df.at[index, 'batter'] = batter_full_name
        else:
            print(f"Batter short name '{batter_short_name}' not found for Match ID: {match_id}, Batting Team: {batting_team}")
        
        if bowler_short_name in bowl_squad_row['short name']:
            bowler_full_name = bowl_squad_row['full name'][bowl_squad_row['short name'].index(bowler_short_name)]
            merged_df.at[index, 'bowler'] = bowler_full_name
        else:
            print(f"Bowler short name '{bowler_short_name}' not found for Match ID: {match_id}, Bowling Team: {bowling_team}")
    else:
        print(f"No matching squad information found for Match ID: {match_id}, Batting Team: {batting_team}, Bowling Team: {bowling_team}")

# Display the updated merged_df
merged_df.head()

KeyError: 'Match ID'

In [72]:
final_list_df = merged_df

# Fill missing values in 'extras_runs' column with 0
final_list_df['extras_runs'] = final_list_df['extras_runs'].fillna(0)
final_list_df['total_runs'] = final_list_df['total_runs'].fillna(0)
final_list_df['batter_runs'] = final_list_df['batter_runs'].fillna(0)

# Convert 'extras_runs' column to integers
final_list_df['extras_runs'] = final_list_df['extras_runs'].astype(int)
final_list_df['total_runs'] = final_list_df['total_runs'].astype(int)
final_list_df['batter_runs'] = final_list_df['batter_runs'].astype(int)

final_list_df.tail()

,Match ID,inning_number,batting_team,bowling_team,over,delivery,batter,bowler,total_runs,batter_runs,extras_runs,extras_type,wickets,bowler_wicket,player_out,wicket_type
479540,2214,2,Mrisho Primary Girls,Baptist Girls,14,2,Ester Kusekwa,Rose Sheko,2,2,0,NaN,0,0,NaN,NaN
479541,2214,2,Mrisho Primary Girls,Baptist Girls,14,3,Ester Kusekwa,Rose Sheko,0,0,0,NaN,0,0,NaN,NaN
479542,2214,2,Mrisho Primary Girls,Baptist Girls,14,4,Ester Kusekwa,Rose Sheko,0,0,0,NaN,0,0,NaN,NaN
479543,2214,2,Mrisho Primary Girls,Baptist Girls,14,5,Ester Kusekwa,Rose Sheko,0,0,0,NaN,0,0,NaN,NaN
479544,2214,2,Mrisho Primary Girls,Baptist Girls,14,6,Ester Kusekwa,Rose Sheko,0,0,0,NaN,1,1,Ester Kusekwa,caught


In [73]:
final_list_df.to_csv("final_list_df.csv", index=False)

In [74]:
final_list_df = pd.read_csv("final_list_df.csv")
final_list_df.head()

,Match ID,inning_number,batting_team,bowling_team,over,delivery,batter,bowler,total_runs,batter_runs,extras_runs,extras_type,wickets,bowler_wicket,player_out,wicket_type
0,2,1,Aksc,Gymkhana,1,1,S Halari,A Nagewadia,0,0,0,NaN,0,0,NaN,NaN
1,2,1,Aksc,Gymkhana,1,2,S Halari,A Nagewadia,0,0,0,NaN,0,0,NaN,NaN
2,2,1,Aksc,Gymkhana,1,3,S Halari,A Nagewadia,0,0,0,NaN,0,0,NaN,NaN
3,2,1,Aksc,Gymkhana,1,4,S Halari,A Nagewadia,0,0,0,NaN,0,0,NaN,NaN
4,2,1,Aksc,Gymkhana,1,5,S Halari,A Nagewadia,2,2,0,NaN,0,0,NaN,NaN


### Replacing Short Names of Batters and Bowlers with Full Names (Comes into the attack method)

In [37]:
chunk_size = 100000  # Define the size of each chunk

# Calculate the total number of chunks needed
num_chunks = (len(merged_df) - 1) // chunk_size + 1

for chunk_index in range(num_chunks):
    start_index = chunk_index * chunk_size
    end_index = min((chunk_index + 1) * chunk_size, len(merged_df))

    # Get the current chunk
    current_chunk = merged_df.iloc[start_index:end_index]

    # Iterate over the rows of the current chunk
    for index, row in current_chunk.iterrows():
        # Check if the batter's short name is not None and matches any short name in full_names_df
        if row['batter'] is not None:
            full_name = full_names_df.loc[(full_names_df['Match ID'] == row['Match ID']) & (full_names_df['short name'] == row['batter']), 'full name'].values
            if len(full_name) > 0:
                # Replace the batter's name with the full name
                merged_df.at[index, 'batter'] = full_name[0]

        # Check if the bowler's short name is not None and matches any short name in full_names_df
        if row['bowler'] is not None:
            full_name = full_names_df.loc[(full_names_df['Match ID'] == row['Match ID']) & (full_names_df['short name'] == row['bowler']), 'full name'].values
            if len(full_name) > 0:
                # Replace the bowler's name with the full name
                merged_df.at[index, 'bowler'] = full_name[0]

    # Display progress or save the chunk to a file
    print(f"Processed chunk {chunk_index + 1}/{num_chunks}")

    # Optionally, save the chunk to a file
    current_chunk.to_csv(f"merged_df_chunk_{chunk_index + 1}.csv", index=False)

print("Processing complete!")

Processed chunk 1/5
Processed chunk 2/5
Processed chunk 3/5
Processed chunk 4/5
Processed chunk 5/5
Processing complete!


In [38]:
chunk_1 = pd.read_csv("merged_df_chunk_1.csv")
chunk_2 = pd.read_csv("merged_df_chunk_2.csv")
chunk_3 = pd.read_csv("merged_df_chunk_3.csv")
chunk_4 = pd.read_csv("merged_df_chunk_4.csv")
chunk_5 = pd.read_csv("merged_df_chunk_5.csv")

In [41]:
general_info_df = pd.concat([chunk_1, chunk_2, chunk_3, chunk_4, chunk_5])

# Fill missing values in 'extras_runs' column with 0
general_info_df['extras_runs'] = general_info_df['extras_runs'].fillna(0)

# Convert 'extras_runs' column to integers
general_info_df['extras_runs'] = general_info_df['extras_runs'].astype(int)

general_info_df.head()

,Match ID,inning_number,batting_team,bowling_team,over,delivery,batter,bowler,total_runs,batter_runs,extras_runs,extras_type,wickets,bowler_wicket,player_out,wicket_type
0,2,1,Aksc,Gymkhana,1,1,S Halari,A Nagewadia,0.0,0.0,0,NaN,0,0,NaN,NaN
1,2,1,Aksc,Gymkhana,1,2,S Halari,A Nagewadia,0.0,0.0,0,NaN,0,0,NaN,NaN
2,2,1,Aksc,Gymkhana,1,3,S Halari,A Nagewadia,0.0,0.0,0,NaN,0,0,NaN,NaN
3,2,1,Aksc,Gymkhana,1,4,S Halari,A Nagewadia,0.0,0.0,0,NaN,0,0,NaN,NaN
4,2,1,Aksc,Gymkhana,1,5,S Halari,A Nagewadia,2.0,2.0,0,NaN,0,0,NaN,NaN


In [42]:
general_info_df.to_csv("final_ball_by_ball.csv", index=False)

## 2. Match Details (general_info)

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

def scrape_match_data(match_id):
    url = f'https://www.cricclubs.com/Tanzania/fullScorecard.do?matchId={match_id}&clubId=7605'
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the table
        table = soup.find('div', class_='match-detail-table')
        if table:
            table = table.find('table')
        else:
            print(f"No match detail table found for match ID {match_id}")
            return None

        match_data = {}

        for tr in table.find_all('tr'):
            row = [th.text.strip() for th in tr.find_all('th')]
            if row:
                # Ensure each row contains exactly two elements (key, value)
                if len(row) == 2:
                    match_data[row[0]] = row[1]

        h3_element = soup.find('h3')
        h3_text = h3_element.text.strip()
        match_data['Match Result'] = h3_text

        team_name_elements = soup.find_all(class_='teamName')
        team_names = [element.get_text(strip=True) for element in team_name_elements]
        match_data['Teams'] = f"{team_names[0]}, {team_names[1]}"

        # Find the element with class 'win'
        winner_elements = soup.find_all(class_='win')

        # Check if there is more than one winner
        if len(winner_elements) > 1:
            winner_team_name = "Tie"
        else:
            winner_element = winner_elements[0]
            winner_team_name = winner_element.find(class_='teamName').get_text(strip=True)

        # Assign the winner to the 'Winner' column in the DataFrame
        match_data['Winner'] = winner_team_name

        overs_tags = soup.find_all('p', style='text-transform: lowercase;')

        overs_played = []
        avail_overs = []

        for overs_tag in overs_tags:
            overs_text = overs_tag.get_text(strip=True)
            overs_split = overs_text.split()
            if len(overs_split) >= 2:
                overs = overs_split[0]
                available_overs = overs_split[1]
                cleaned_avail_overs = re.sub(r'[^0-9]', '', available_overs)

                overs_played.append(overs)
                avail_overs.append(cleaned_avail_overs)
            else:
                print("Invalid format for overs data:", overs_text)

        if overs_played:
            match_data["first_bat_overs"] = overs_played[0]
        if avail_overs:
            match_data["first_bat_avail_overs"] = avail_overs[0]
        if len(overs_played) > 1:
            match_data["second_bat_overs"] = overs_played[1]
        if len(avail_overs) > 1:
            match_data["second_bat_avail_overs"] = avail_overs[1]

        match_data["match_id"] = match_id
        
        return match_data
    else:
        print(f"Failed to retrieve data for match ID {match_id}")
        return None

# Example usage:
# match_data = scrape_match_data("2000")
# print(match_data)

In [16]:
# Initialize an empty list to store match data dictionaries
all_match_data = []

# Set the initial match ID
match_id = 2121

# Counter to track consecutive failed attempts
failed_attempts = 0

# Maximum consecutive failed attempts allowed
max_failed_attempts = 5

while True:
    # Scrape match data for the current match ID
    match_data = scrape_match_data(match_id)
    
    # Check if match_data is not None (i.e., match data was successfully scraped)
    if match_data is not None:
        # Append the match data dictionary to the list of all match data dictionaries
        all_match_data.append(match_data)
        
        # Reset the counter of consecutive failed attempts
        failed_attempts = 0
        
        # Increment the match ID for the next iteration
        match_id += 1
    else:
        # Increment the counter of consecutive failed attempts
        failed_attempts += 1
        
        # Check if the maximum consecutive failed attempts limit has been reached
        if failed_attempts >= max_failed_attempts:
            # Break the loop if the limit has been reached
            break
        else:
            # Increment the match ID for the next iteration
            match_id += 1

# Convert the list of match data dictionaries into a DataFrame
general_info_df = pd.DataFrame(all_match_data)

# Display the final DataFrame
general_info_df.head()

No match detail table found for match ID 2123
No match detail table found for match ID 2140
No match detail table found for match ID 2149
No match detail table found for match ID 2187
No match detail table found for match ID 2331
No match detail table found for match ID 2332
No match detail table found for match ID 2333
No match detail table found for match ID 2334
No match detail table found for match ID 2335


,Topic,Series:,Match Date:,Toss:,Player of the Match:,Location:,Points Earned:,1st Innings:,Innings break:,2nd Innings:,Last Updated:,Match Documents:,Match Result,Teams,Winner,first_bat_overs,first_bat_avail_overs,second_bat_overs,second_bat_avail_overs,match_id
0,Details,TCA JUNIOR LEAGUE 2024 - GEITA BOYS,19/04/2024,Mbugani Boys\r\n ...,Anord Silvanus,TBD,Ukombozi Boys\r\n ...,71 min 3:09 PM 4:21 PM,4 min 4:21 PM 4:25 PM,45 min 4:25 PM 5:11 PM,Yalinde Maurice\r\n ...,,TCA JUNIOR LEAGUE 2024 - GEITA BOYS: League\r...,"Ukombozi Boys, Mbugani Boys",Mbugani Boys,20.0,20,14.3,20,2121
1,Details,TCA JUNIOR LEAGUE 2024 - DODOMA MJINI BOYS,20/04/2024,Kaloleni Boys\r\n ...,Simon Hassani,Dodoma Jiji,Kaloleni Boys\r\n ...,62 min 1:49 PM 2:52 PM,12 min 2:52 PM 3:04 PM,37 min 3:04 PM 3:41 PM,Benson Nyaikini\r\n ...,,TCA JUNIOR LEAGUE 2024 - DODOMA MJINI BOYS: L...,"Kaloleni Boys, Nkunhungu Boys",Nkunhungu Boys,14.4,20,10.2,20,2122
2,Details,TCA JUNIOR LEAGUE 2024 - ARUSHA GIRLS,20/04/2024,Themi Girls\r\n ...,Brenda Samwel,TBD,Wema Girls\r\n ...,48 min 09:07 AM 09:56 AM,3 min 09:56 AM 09:59 AM,47 min 09:59 AM 10:47 AM,Kelvin Njilinji\r\n ...,,TCA JUNIOR LEAGUE 2024 - ARUSHA GIRLS: League...,"Wema Girls, Themi Girls",Tie,12.1,20,9.5,20,2124
3,Details,TCA JUNIOR LEAGUE 2024 - ARUSHA GIRLS,20/04/2024,Lemara Girls\r\n ...,Megan Brayson,TBD,Engira Girls\r\n ...,40 min 09:12 AM 09:52 AM,8 min 09:52 AM 10:01 AM,42 min 10:01 AM 10:44 AM,Nasibu Kelvin Mapunda\r\n ...,,TCA JUNIOR LEAGUE 2024 - ARUSHA GIRLS: League...,"Engira Girls, Lemara Girls",Lemara Girls,11.3,20,13.2,20,2125
4,Details,TCA JUNIOR LEAGUE 2024 - GAIRO GIRLS,20/04/2024,Gairo C Girls\r\n ...,Joyce Emmanuel,Gairo,Gairo C Girls\r\n ...,48 min 09:12 AM 10:01 AM,13 min 10:01 AM 10:14 AM,46 min 10:14 AM 11:01 AM,Nuru Omary\r\n ...,,TCA JUNIOR LEAGUE 2024 - GAIRO GIRLS: League\...,"Gairo C Girls, Lolela Girls",Gairo C Girls,20.0,20,12.5,20,2126


In [117]:
general_info_df.to_csv("general_info_df.csv", index=False)

In [168]:
general_info_df = pd.read_csv("general_info_df.csv")
general_info_df.tail(1)

,Topic,Series:,Match Date:,Toss:,Player of the Match:,Location:,Points Earned:,1st Innings:,Innings break:,2nd Innings:,...,Match Documents:,Match Result,Teams,Winner,first_bat_overs,first_bat_avail_overs,second_bat_overs,second_bat_avail_overs,match_id,1st Innings Start:
2184,Details,TCA JUNIOR LEAGUE 2024 - TANGA GIRLS,06/05/2024,Chuda Girls\r\n ...,Fatuma Mussa,USAGARA,Mwenge Girls\r\n ...,57 min 3:35 PM 4:32 PM,10 min 4:32 PM 4:42 PM,29 min 4:42 PM 5:12 PM,...,NaN,TCA JUNIOR LEAGUE 2024 - TANGA GIRLS: League\...,"Mwenge Girls, Chuda Girls",Chuda Girls,10.5,20.0,8.3,20.0,2246,NaN


#### Cleaning the Dataframe

In [17]:
# Check for empty or missing data in columns
empty_columns = general_info_df.columns[general_info_df.isnull().any()]
print(empty_columns)

# Fill missing values in the "Points Earned" column with 0
general_info_df['Points Earned:'].fillna("", inplace=True)

Index(['Player of the Match:'], dtype='object')


In [18]:
# Remove '\r\n' and extra whitespace from all columns
general_info_df = general_info_df.apply(lambda x: x.str.replace('\r\n', '').str.strip() if x.dtype == 'object' else x)

# Remove any leading and trailing whitespace from all columns
general_info_df = general_info_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

# Convert 'Match Date' column to datetime format
general_info_df['Match Date:'] = pd.to_datetime(general_info_df['Match Date:'], format='%d/%m/%Y', errors='coerce')

# Drop any rows with missing values
# general_info_df = general_info_df.dropna()

# Clean columns and remove ':' symbol
general_info_df.columns = general_info_df.columns.str.replace(':', '')

# Columns to clean
columns_to_clean = ['Points Earned:', 'Last Updated:', 'Match Result:', 'Toss:']

# Loop through each column and clean the values
for column in columns_to_clean:
    # Check if the column exists in the DataFrame
    if column in general_info_df.columns:
        # Apply cleaning operation to all values in the column
        general_info_df[column] = general_info_df[column].str.replace('\r\n', '').str.strip()
        general_info_df[column] = general_info_df[column].str.replace(r'\s+', ' ', regex=True)

# Clean the 'Match Result' column
general_info_df['Match Result'] = general_info_df['Match Result'].apply(lambda x: x.split('-', 2)[-1].strip())
# Remove the date from the "Match Result" column
general_info_df['Match Result'] = general_info_df['Match Result'].apply(lambda x: re.sub(r'\(\d{2}/\d{2}/\d{4}\)', '', x).strip())

general_info_df = general_info_df.drop(columns=['Topic'])

general_info_df = general_info_df.rename(columns={'Location': 'venue', 'Match Date': 'dates', 'Series': 'event_name', 
                        'Player of the Match': 'player_of_match'})

In [19]:
# Define a function to clean the innings information
def clean_innings_info(innings_str):
    if isinstance(innings_str, str):
        # Define regular expression pattern to extract time and duration
        pattern = r'(\d{1,3} min)\s*(\d{1,2}:\d{2} [AP]M)\s*(\d{1,2}:\d{2} [AP]M)'
        
        # Match the pattern in the innings string
        match = re.match(pattern, innings_str)
        
        if match:
            duration = match.group(1)
            start_time = match.group(2)
            end_time = match.group(3)
            
            # Format the result as a dictionary
            result = {'Start': start_time, 'End': end_time, 'Duration': duration}
            return result
        else:
            return None
    else:
        return {'Start': np.nan, 'End': np.nan, 'Duration': np.nan}

# Apply the function to clean the '1st Innings', 'Innings break', and '2nd Innings' columns
general_info_df['1st Innings'] = general_info_df['1st Innings'].apply(clean_innings_info)
general_info_df['Innings break'] = general_info_df['Innings break'].apply(clean_innings_info)
general_info_df['2nd Innings'] = general_info_df['2nd Innings'].apply(clean_innings_info)

general_info_df.head(1)

,event_name,dates,Toss,player_of_match,venue,Points Earned,1st Innings,Innings break,2nd Innings,Last Updated,Match Documents,Match Result,Teams,Winner,first_bat_overs,first_bat_avail_overs,second_bat_overs,second_bat_avail_overs,match_id
0,TCA JUNIOR LEAGUE 2024 - GEITA BOYS,2024-04-19,Mbugani Boys ...,Anord Silvanus,TBD,Ukombozi Boys ...,"{'Start': '3:09 PM', 'End': '4:21 PM', 'Durati...","{'Start': '4:21 PM', 'End': '4:25 PM', 'Durati...","{'Start': '4:25 PM', 'End': '5:11 PM', 'Durati...",Yalinde Maurice ...,,Mbugani Boys won by 2 Wickets,"Ukombozi Boys, Mbugani Boys",Mbugani Boys,20.0,20,14.3,20,2121


In [20]:
# Define a function to extract the team name and toss decision
def extract_toss_info(toss_str, teams_str):
    # Extract team names from the 'Teams' column
    teams = teams_str.split(', ')
    team_names = [team.strip() for team in teams]
    
    # Define regular expression for the toss decision
    toss_decision_pattern = r'(?:elected to )?(bat|bowl)'
    
    # Extract team name
    toss_winner = None
    for team_name in team_names:
        if team_name.lower() in toss_str.lower():
            toss_winner = team_name
            break
    
    # Extract toss decision
    toss_decision = re.search(toss_decision_pattern, toss_str, re.IGNORECASE)
    if toss_decision:
        toss_decision = toss_decision.group(1)
    else:
        toss_decision = 'Unknown'
    
    return toss_winner, toss_decision

# Apply the function to the 'Toss' column to create new columns 'Toss Winner' and 'Toss Decision'
general_info_df[['Toss Winner', 'Toss Decision']] = general_info_df.apply(lambda x: pd.Series(extract_toss_info(x['Toss'], x['Teams'])), axis=1)

# Add 2 new columns based on the value of "Toss Decision"
general_info_df['first_bat'] = general_info_df.apply(lambda x: x['Toss Winner'] if x['Toss Decision'].lower() == 'bat' else [team for team in x['Teams'].split(', ') if team != x['Toss Winner']][0] if x['Toss Decision'].lower() == 'bowl' else None, axis=1)
general_info_df['second_bat'] = general_info_df.apply(lambda x: [team for team in x['Teams'].split(', ') if team != x['Toss Winner']][0] if x['Toss Decision'].lower() == 'bat' else x['Toss Winner'] if x['Toss Decision'].lower() == 'bowl' else None, axis=1)

# Clean the "Match Result" column to remove parentheses and the string indicating no result
general_info_df['Match Result'] = general_info_df['Match Result'].str.replace(r'\([^()]*\)', '').str.replace(r' - No Result.*', '', regex=True)

# Split the "Match Result" column into two columns: "Margin" and "By" only if there is a match result
if 'Match Result' in general_info_df.columns and general_info_df['Match Result'].notnull().any():
    result_pattern = r'(\d+) (Run|Wicket)s?'
    match_result_split = general_info_df['Match Result'].str.extract(result_pattern)
    if match_result_split is not None:
        general_info_df[['Margin', 'By']] = match_result_split
    else:
        general_info_df[['Margin', 'By']] = np.nan, np.nan
else:
    general_info_df['Margin'] = np.nan
    general_info_df['By'] = np.nan

# Adjust the "By" column to display "Runs" or "Wickets"
general_info_df['By'] = general_info_df['By'].apply(lambda x: x + 's' if isinstance(x, str) and x == 'Run' else str(x) + 's')

# Check if the match result indicates a tie and append null values to "Margin" and "By" columns
general_info_df.loc[general_info_df['Match Result'].str.contains('Tie'), ['Margin', 'By']] = np.nan, np.nan

#general_info_df = general_info_df.drop(columns=['Toss', 'Match Result'])

# Display the DataFrame with new columns
general_info_df.head(1)

,event_name,dates,Toss,player_of_match,venue,Points Earned,1st Innings,Innings break,2nd Innings,Last Updated,...,first_bat_avail_overs,second_bat_overs,second_bat_avail_overs,match_id,Toss Winner,Toss Decision,first_bat,second_bat,Margin,By
0,TCA JUNIOR LEAGUE 2024 - GEITA BOYS,2024-04-19,Mbugani Boys ...,Anord Silvanus,TBD,Ukombozi Boys ...,"{'Start': '3:09 PM', 'End': '4:21 PM', 'Durati...","{'Start': '4:21 PM', 'End': '4:25 PM', 'Durati...","{'Start': '4:25 PM', 'End': '5:11 PM', 'Durati...",Yalinde Maurice ...,...,20,14.3,20,2121,Mbugani Boys,bowl,Ukombozi Boys,Mbugani Boys,2,Wickets


### For General Use

In [21]:
# 1. Create a new DataFrame with desired columns
new_df = general_info_df[['dates', 'match_id', 'event_name', 'venue', 'Last Updated', 'first_bat', 'first_bat_overs', 'first_bat_avail_overs', 'second_bat', 'second_bat_overs', 'second_bat_avail_overs', 'Winner', 'Margin', 'By', 'player_of_match', '1st Innings', 'Innings break', '2nd Innings']].copy()

# 2. Split the "1st Innings" column by keys into separate columns
first_innings_info = new_df['1st Innings'].apply(pd.Series)

# 3. Rename the new columns with prefix
first_innings_info = first_innings_info.add_prefix('1st Inns ')

# 4. Concatenate the new columns with the original DataFrame
new_df = pd.concat([new_df, first_innings_info], axis=1)

break_innings_info = new_df['Innings break'].apply(pd.Series)
break_innings_info = break_innings_info.add_prefix('Inns break ')
new_df = pd.concat([new_df, break_innings_info], axis=1)

second_innings_info = new_df['2nd Innings'].apply(pd.Series)
second_innings_info = second_innings_info.add_prefix('2nd Inns ')
new_df = pd.concat([new_df, second_innings_info], axis=1)

# Drop the original "1st Innings" column
new_df = new_df.drop(columns=['1st Innings'])
new_df = new_df.drop(columns=['2nd Innings'])
new_df = new_df.drop(columns=['Innings break'])

new_df.head(1)

,dates,match_id,event_name,venue,Last Updated,first_bat,first_bat_overs,first_bat_avail_overs,second_bat,second_bat_overs,...,player_of_match,1st Inns Start,1st Inns End,1st Inns Duration,Inns break Start,Inns break End,Inns break Duration,2nd Inns Start,2nd Inns End,2nd Inns Duration
0,2024-04-19,2121,TCA JUNIOR LEAGUE 2024 - GEITA BOYS,TBD,Yalinde Maurice ...,Ukombozi Boys,20.0,20,Mbugani Boys,14.3,...,Anord Silvanus,3:09 PM,4:21 PM,71 min,4:21 PM,4:25 PM,4 min,4:25 PM,5:11 PM,45 min


In [22]:
# Split "Last Updated" column into "Scorer" and "Last Update Time"
new_df[['Scorer', 'Last Update Time']] = new_df['Last Updated'].str.extract(r'^(.*?) \((.*?)\)$')

# Drop the original "Last Updated" column
new_df = new_df.drop(columns=['Last Updated'])

# Display the updated DataFrame
new_df.head()

,dates,match_id,event_name,venue,first_bat,first_bat_overs,first_bat_avail_overs,second_bat,second_bat_overs,second_bat_avail_overs,...,1st Inns End,1st Inns Duration,Inns break Start,Inns break End,Inns break Duration,2nd Inns Start,2nd Inns End,2nd Inns Duration,Scorer,Last Update Time
0,2024-04-19,2121,TCA JUNIOR LEAGUE 2024 - GEITA BOYS,TBD,Ukombozi Boys,20.0,20,Mbugani Boys,14.3,20,...,4:21 PM,71 min,4:21 PM,4:25 PM,4 min,4:25 PM,5:11 PM,45 min,Yalinde Maurice ...,19/04/2024 17:11:00 ...
1,2024-04-20,2122,TCA JUNIOR LEAGUE 2024 - DODOMA MJINI BOYS,Dodoma Jiji,Kaloleni Boys,14.4,20,Nkunhungu Boys,10.2,20,...,2:52 PM,62 min,2:52 PM,3:04 PM,12 min,3:04 PM,3:41 PM,37 min,Benson Nyaikini ...,20/04/2024 15:42:00 ...
2,2024-04-20,2124,TCA JUNIOR LEAGUE 2024 - ARUSHA GIRLS,TBD,Wema Girls,12.1,20,Themi Girls,9.5,20,...,09:56 AM,48 min,09:56 AM,09:59 AM,3 min,09:59 AM,10:47 AM,47 min,Kelvin Njilinji ...,20/04/2024 10:48:00 ...
3,2024-04-20,2125,TCA JUNIOR LEAGUE 2024 - ARUSHA GIRLS,TBD,Engira Girls,11.3,20,Lemara Girls,13.2,20,...,09:52 AM,40 min,09:52 AM,10:01 AM,8 min,10:01 AM,10:44 AM,42 min,Nasibu Kelvin Mapunda ...,20/04/2024 10:44:00 ...
4,2024-04-20,2126,TCA JUNIOR LEAGUE 2024 - GAIRO GIRLS,Gairo,Gairo C Girls,20.0,20,Lolela Girls,12.5,20,...,10:01 AM,48 min,10:01 AM,10:14 AM,13 min,10:14 AM,11:01 AM,46 min,Nuru Omary ...,20/04/2024 11:02:00 ...


In [27]:
new_df.describe()

,dates,match_id
count,206,206.000000
mean,2024-05-04 11:18:03.495145728,2226.970874
min,2024-04-19 00:00:00,2121.000000
25%,2024-04-21 00:00:00,2175.250000
50%,2024-05-04 00:00:00,2227.500000
75%,2024-05-16 00:00:00,2278.750000
max,2024-05-26 00:00:00,2330.000000
std,NaN,60.329818


In [28]:
# Filter the DataFrame to include only rows where 'event_name' contains 'JUNIOR'
new_df = new_df[new_df['event_name'].str.contains('JUNIOR')]

# Display the filtered DataFrame
new_df.head()

,dates,match_id,event_name,venue,first_bat,first_bat_overs,first_bat_avail_overs,second_bat,second_bat_overs,second_bat_avail_overs,...,1st Inns End,1st Inns Duration,Inns break Start,Inns break End,Inns break Duration,2nd Inns Start,2nd Inns End,2nd Inns Duration,Scorer,Last Update Time
0,2024-04-19,2121,TCA JUNIOR LEAGUE 2024 - GEITA BOYS,TBD,Ukombozi Boys,20.0,20,Mbugani Boys,14.3,20,...,4:21 PM,71 min,4:21 PM,4:25 PM,4 min,4:25 PM,5:11 PM,45 min,Yalinde Maurice ...,19/04/2024 17:11:00 ...
1,2024-04-20,2122,TCA JUNIOR LEAGUE 2024 - DODOMA MJINI BOYS,Dodoma Jiji,Kaloleni Boys,14.4,20,Nkunhungu Boys,10.2,20,...,2:52 PM,62 min,2:52 PM,3:04 PM,12 min,3:04 PM,3:41 PM,37 min,Benson Nyaikini ...,20/04/2024 15:42:00 ...
2,2024-04-20,2124,TCA JUNIOR LEAGUE 2024 - ARUSHA GIRLS,TBD,Wema Girls,12.1,20,Themi Girls,9.5,20,...,09:56 AM,48 min,09:56 AM,09:59 AM,3 min,09:59 AM,10:47 AM,47 min,Kelvin Njilinji ...,20/04/2024 10:48:00 ...
3,2024-04-20,2125,TCA JUNIOR LEAGUE 2024 - ARUSHA GIRLS,TBD,Engira Girls,11.3,20,Lemara Girls,13.2,20,...,09:52 AM,40 min,09:52 AM,10:01 AM,8 min,10:01 AM,10:44 AM,42 min,Nasibu Kelvin Mapunda ...,20/04/2024 10:44:00 ...
4,2024-04-20,2126,TCA JUNIOR LEAGUE 2024 - GAIRO GIRLS,Gairo,Gairo C Girls,20.0,20,Lolela Girls,12.5,20,...,10:01 AM,48 min,10:01 AM,10:14 AM,13 min,10:14 AM,11:01 AM,46 min,Nuru Omary ...,20/04/2024 11:02:00 ...


In [30]:
new_df.to_excel("all_jr_league_matches.xlsx", index=False)

### For Audit:

In [8]:
# 1. Create a new DataFrame with desired columns
new_df = general_info_df[['dates', 'match_id', 'event_name', '1st Innings', 'Innings break', '2nd Innings', 'Last Updated', 'first_bat', 'first_bat_overs', 'first_bat_avail_overs', 'second_bat', 'second_bat_overs', 'second_bat_avail_overs', 'Winner']].copy()

# 2. Split the "1st Innings" column by keys into separate columns
first_innings_info = new_df['1st Innings'].apply(pd.Series)

# 3. Rename the new columns with prefix
first_innings_info = first_innings_info.add_prefix('1st Inns ')

# 4. Concatenate the new columns with the original DataFrame
new_df = pd.concat([new_df, first_innings_info], axis=1)

break_innings_info = new_df['Innings break'].apply(pd.Series)
break_innings_info = break_innings_info.add_prefix('Inns break ')
new_df = pd.concat([new_df, break_innings_info], axis=1)

second_innings_info = new_df['2nd Innings'].apply(pd.Series)
second_innings_info = second_innings_info.add_prefix('2nd Inns ')
new_df = pd.concat([new_df, second_innings_info], axis=1)

# Drop the original "1st Innings" column
new_df = new_df.drop(columns=['1st Innings'])
new_df = new_df.drop(columns=['2nd Innings'])
new_df = new_df.drop(columns=['Innings break'])

new_df.head()

,dates,match_id,event_name,Last Updated,first_bat,first_bat_overs,first_bat_avail_overs,second_bat,second_bat_overs,second_bat_avail_overs,Winner,1st Inns Start,1st Inns End,1st Inns Duration,Inns break Start,Inns break End,Inns break Duration,2nd Inns Start,2nd Inns End,2nd Inns Duration
0,2024-05-24,2313,TCA JUNIOR LEAGUE 2024 - CHAMWINO GIRLS,Ally Mpeka ...,Kambarage B Girls,20.0,20,Mkapa B Girls,18.3,20,Kambarage B Girls,1:16 PM,2:28 PM,72 min,2:28 PM,2:42 PM,14 min,2:42 PM,3:41 PM,58 min
1,2024-05-24,2314,TCA JUNIOR LEAGUE 2024 - CHAMWINO GIRLS,Ally Mpeka ...,Chamwino Secondary A Girls,20.0,20,Mkapa C Girls,13.1,20,Chamwino Secondary A Girls,3:58 PM,5:04 PM,66 min,5:04 PM,5:14 PM,9 min,5:14 PM,6:42 PM,87 min
2,2024-05-25,2315,TCA JUNIOR LEAGUE 2024 - MOROGORO GIRLS,Kassimu Nassoro ...,Mafiga B Girls,15.5,20,Kambarage Girls,3.5,20,Kambarage Girls,09:12 AM,09:54 AM,42 min,09:54 AM,10:05 AM,10 min,10:05 AM,10:14 AM,8 min
3,2024-05-25,2316,TCA JUNIOR LEAGUE 2024 - GAIRO GIRLS,Sefu Khalifa ...,Bwawani Girls,14.5,20,Ibuti Girls,7.3,20,Ibuti Girls,09:07 AM,09:59 AM,52 min,09:59 AM,10:17 AM,17 min,10:17 AM,10:41 AM,24 min
4,2024-05-25,2317,TCA JUNIOR LEAGUE 2024 - MOROGORO GIRLS,Kassimu Nassoro ...,Mtawala Girls,7.0,20,Kambarage Girls,4.1,20,Kambarage Girls,11:08 AM,11:34 AM,25 min,11:34 AM,11:42 AM,8 min,11:42 AM,11:57 AM,14 min


In [9]:
# Convert over notation to decimal notation for "first_bat_overs" column
new_df['first_bat_overs_decimal'] = new_df['first_bat_overs'].apply(lambda x: int(x.split('.')[0]) + int(x.split('.')[1])/6)

# Remove rows with NaN values in "first_bat_overs" column
new_df = new_df.dropna(subset=['first_bat_overs'])

# Convert the "1st Inns Duration" column to minutes
new_df['1st Inns Duration'] = pd.to_timedelta(new_df['1st Inns Duration']).dt.total_seconds() / 60

# Calculate the "1st Inns Mins Per Over" by dividing "1st Inns Duration" by "first_bat_overs_decimal"
new_df['1st Inns Mins Per Over'] = new_df['1st Inns Duration'] / new_df['first_bat_overs_decimal']

# Drop the temporary column
new_df = new_df.drop(columns=['first_bat_overs_decimal'])

# Convert over notation to decimal notation for "second_bat_overs" column
new_df['second_bat_overs_decimal'] = new_df['second_bat_overs'].apply(lambda x: int(x.split('.')[0]) + int(x.split('.')[1])/6)

# Remove rows with NaN values in "second_bat_overs" column
new_df = new_df.dropna(subset=['second_bat_overs'])

# Convert the "2nd Inns Duration" column to minutes
new_df['2nd Inns Duration'] = pd.to_timedelta(new_df['2nd Inns Duration']).dt.total_seconds() / 60

# Calculate the "2nd Inns Mins Per Over" by dividing "2nd Inns Duration" by "second_bat_overs_decimal"
new_df['2nd Inns Mins Per Over'] = new_df['2nd Inns Duration'] / new_df['second_bat_overs_decimal']

# Drop the temporary column
new_df = new_df.drop(columns=['second_bat_overs_decimal'])

# Display the new DataFrame
new_df.head()

,dates,match_id,event_name,Last Updated,first_bat,first_bat_overs,first_bat_avail_overs,second_bat,second_bat_overs,second_bat_avail_overs,...,1st Inns End,1st Inns Duration,Inns break Start,Inns break End,Inns break Duration,2nd Inns Start,2nd Inns End,2nd Inns Duration,1st Inns Mins Per Over,2nd Inns Mins Per Over
0,2024-05-24,2313,TCA JUNIOR LEAGUE 2024 - CHAMWINO GIRLS,Ally Mpeka ...,Kambarage B Girls,20.0,20,Mkapa B Girls,18.3,20,...,2:28 PM,72.0,2:28 PM,2:42 PM,14 min,2:42 PM,3:41 PM,58.0,3.600000,3.135135
1,2024-05-24,2314,TCA JUNIOR LEAGUE 2024 - CHAMWINO GIRLS,Ally Mpeka ...,Chamwino Secondary A Girls,20.0,20,Mkapa C Girls,13.1,20,...,5:04 PM,66.0,5:04 PM,5:14 PM,9 min,5:14 PM,6:42 PM,87.0,3.300000,6.607595
2,2024-05-25,2315,TCA JUNIOR LEAGUE 2024 - MOROGORO GIRLS,Kassimu Nassoro ...,Mafiga B Girls,15.5,20,Kambarage Girls,3.5,20,...,09:54 AM,42.0,09:54 AM,10:05 AM,10 min,10:05 AM,10:14 AM,8.0,2.652632,2.086957
3,2024-05-25,2316,TCA JUNIOR LEAGUE 2024 - GAIRO GIRLS,Sefu Khalifa ...,Bwawani Girls,14.5,20,Ibuti Girls,7.3,20,...,09:59 AM,52.0,09:59 AM,10:17 AM,17 min,10:17 AM,10:41 AM,24.0,3.505618,3.200000
4,2024-05-25,2317,TCA JUNIOR LEAGUE 2024 - MOROGORO GIRLS,Kassimu Nassoro ...,Mtawala Girls,7.0,20,Kambarage Girls,4.1,20,...,11:34 AM,25.0,11:34 AM,11:42 AM,8 min,11:42 AM,11:57 AM,14.0,3.571429,3.360000


In [10]:
# Split "Last Updated" column into "Scorer" and "Last Update Time"
new_df[['Scorer', 'Last Update Time']] = new_df['Last Updated'].str.extract(r'^(.*?) \((.*?)\)$')

# Drop the original "Last Updated" column
new_df = new_df.drop(columns=['Last Updated'])

# Display the updated DataFrame
new_df.head()

,dates,match_id,event_name,first_bat,first_bat_overs,first_bat_avail_overs,second_bat,second_bat_overs,second_bat_avail_overs,Winner,...,Inns break Start,Inns break End,Inns break Duration,2nd Inns Start,2nd Inns End,2nd Inns Duration,1st Inns Mins Per Over,2nd Inns Mins Per Over,Scorer,Last Update Time
0,2024-05-24,2313,TCA JUNIOR LEAGUE 2024 - CHAMWINO GIRLS,Kambarage B Girls,20.0,20,Mkapa B Girls,18.3,20,Kambarage B Girls,...,2:28 PM,2:42 PM,14 min,2:42 PM,3:41 PM,58.0,3.600000,3.135135,Ally Mpeka ...,24/05/2024 15:45:00 ...
1,2024-05-24,2314,TCA JUNIOR LEAGUE 2024 - CHAMWINO GIRLS,Chamwino Secondary A Girls,20.0,20,Mkapa C Girls,13.1,20,Chamwino Secondary A Girls,...,5:04 PM,5:14 PM,9 min,5:14 PM,6:42 PM,87.0,3.300000,6.607595,Ally Mpeka ...,24/05/2024 18:42:00 ...
2,2024-05-25,2315,TCA JUNIOR LEAGUE 2024 - MOROGORO GIRLS,Mafiga B Girls,15.5,20,Kambarage Girls,3.5,20,Kambarage Girls,...,09:54 AM,10:05 AM,10 min,10:05 AM,10:14 AM,8.0,2.652632,2.086957,Kassimu Nassoro ...,25/05/2024 10:14:00 ...
3,2024-05-25,2316,TCA JUNIOR LEAGUE 2024 - GAIRO GIRLS,Bwawani Girls,14.5,20,Ibuti Girls,7.3,20,Ibuti Girls,...,09:59 AM,10:17 AM,17 min,10:17 AM,10:41 AM,24.0,3.505618,3.200000,Sefu Khalifa ...,25/05/2024 10:42:00 ...
4,2024-05-25,2317,TCA JUNIOR LEAGUE 2024 - MOROGORO GIRLS,Mtawala Girls,7.0,20,Kambarage Girls,4.1,20,Kambarage Girls,...,11:34 AM,11:42 AM,8 min,11:42 AM,11:57 AM,14.0,3.571429,3.360000,Kassimu Nassoro ...,25/05/2024 11:57:00 ...


In [11]:
# Rename existing columns
new_df = new_df.rename(columns={
    '1st Inns Duration': '1st Inns Duration - Mins',
    '2nd Inns Duration': '2nd Inns Duration - Mins'
})

# Clean up the "Inns break Duration" column to extract only the numerical part
new_df['Inns break Duration'] = new_df['Inns break Duration'].str.extract('(\d+)', expand=False)

# Convert the column to numeric type
new_df['Inns break Duration'] = pd.to_numeric(new_df['Inns break Duration'])

In [12]:
# Define a function to determine the doubt level
def get_doubt_level(row):
    if row['1st Inns Mins Per Over'] < 2 or row['2nd Inns Mins Per Over'] < 2 or row['Inns break Duration'] < 6 or row['Inns break Duration'] > 25 or row['1st Inns Mins Per Over'] > 5 or row['2nd Inns Mins Per Over'] > 5:
        return 'High'
    elif pd.Timestamp(row['dates'].strftime('%Y-%m-%d') + ' 17:00:00') <= pd.to_datetime(row['1st Inns Start']) < pd.Timestamp(row['dates'].strftime('%Y-%m-%d') + ' 07:00:00'):
        return 'High'  # If 1st Inns start time is after 5pm
    elif row['1st Inns Mins Per Over'] < 3 or row['2nd Inns Mins Per Over'] < 3 or row['Inns break Duration'] < 10 or row['Inns break Duration'] > 20:
        return 'Medium'
    else:
        return 'None'

# Apply the function to create the "Doubt Level" column
new_df['Doubt Level'] = new_df.apply(get_doubt_level, axis=1)

In [13]:
# Define a function to determine all reasons for doubt
def get_reason(row):
    reasons = []
    if row['Doubt Level'] == 'High':
        if row['1st Inns Mins Per Over'] < 3 and row['2nd Inns Mins Per Over'] < 3:
            reasons.append(f"Both Innings finished too quickly.\n - First Inns had an over rate of {round(row['1st Inns Mins Per Over'], 2)} minutes per over, with {row['first_bat_overs']} overs being played in {int(row['1st Inns Duration - Mins'])} minutes\n - Second Inns had an over rate of {round(row['2nd Inns Mins Per Over'], 2)} minutes per over, with {row['second_bat_overs']} overs being played in {int(row['2nd Inns Duration - Mins'])} minutes")
        elif row['1st Inns Mins Per Over'] < 3:  
            reasons.append(f"1st Innings over rate was too quick: {round(row['1st Inns Mins Per Over'], 2)} minutes per over, with {row['first_bat_overs']} overs being played in {int(row['1st Inns Duration - Mins'])} minutes.")
        elif row['2nd Inns Mins Per Over'] < 3:
            reasons.append(f"2nd Innings over rate was too quick: {round(row['2nd Inns Mins Per Over'], 2)} minutes per over, with {row['second_bat_overs']} overs being played in {int(row['2nd Inns Duration - Mins'])} minutes.")
        elif row['1st Inns Mins Per Over'] > 5 and row['2nd Inns Mins Per Over'] > 5:
            reasons.append(f"Both Innings took too long.\n - First Inns had an over rate of {round(row['1st Inns Mins Per Over'], 2)} minutes per over, with {row['first_bat_overs']} overs being played in {int(row['1st Inns Duration - Mins'])} minutes\n - Second Inns had an over rate of {round(row['2nd Inns Mins Per Over'], 2)} minutes per over, with {row['second_bat_overs']} overs being played in {int(row['2nd Inns Duration - Mins'])} minutes")
        elif row['1st Inns Mins Per Over'] > 5:  
            reasons.append(f"1st Innings over rate was too slow: {round(row['1st Inns Mins Per Over'], 2)} minutes per over, with {row['first_bat_overs']} overs being played in {int(row['1st Inns Duration - Mins'])} minutes.")
        elif row['2nd Inns Mins Per Over'] > 5:
            reasons.append(f"2nd Innings over rate was too slow: {round(row['2nd Inns Mins Per Over'], 2)} minutes per over, with {row['second_bat_overs']} overs being played in {int(row['2nd Inns Duration - Mins'])} minutes.")
        if pd.Timestamp(row['dates'].strftime('%Y-%m-%d') + ' 17:00:00') <= pd.to_datetime(row['1st Inns Start']) < pd.Timestamp(row['dates'].strftime('%Y-%m-%d') + ' 07:00:00'):
            reasons.append(f"Match start time was too late: {row['1st Inns Start']}")  # If 1st Inns start time is between 5pm and 7am
        if row['Inns break Duration'] < 10:
            reasons.append(f"Innings break was too short: Only {row['Inns break Duration']} minutes long.")
        elif row['Inns break Duration'] > 20:
            reasons.append(f"Innings break was too long: {row['Inns break Duration']} minutes long.")
    else:
        return ''

    # Join reasons into a single string with index and line breaks
    if len(reasons) > 1:
        return '\n'.join([f"{i+1}. {reason}" for i, reason in enumerate(reasons)])
    else:
        return '\n'.join(reasons)

# Apply the function to create the "Reason" column
new_df['Exact Problem'] = new_df.apply(get_reason, axis=1)

new_df['Other Reason'] = ''

new_df['Fixture'] = new_df['first_bat'] + ' vs ' + new_df['second_bat']

print(new_df)

        dates  match_id                               event_name  \
0  2024-05-24      2313  TCA JUNIOR LEAGUE 2024 - CHAMWINO GIRLS   
1  2024-05-24      2314  TCA JUNIOR LEAGUE 2024 - CHAMWINO GIRLS   
2  2024-05-25      2315  TCA JUNIOR LEAGUE 2024 - MOROGORO GIRLS   
3  2024-05-25      2316     TCA JUNIOR LEAGUE 2024 - GAIRO GIRLS   
4  2024-05-25      2317  TCA JUNIOR LEAGUE 2024 - MOROGORO GIRLS   
5  2024-05-25      2318      TCA JUNIOR LEAGUE 2024 - TANGA BOYS   
6  2024-05-25      2319     TCA JUNIOR LEAGUE 2024 - GAIRO GIRLS   
7  2024-05-25      2320      TCA JUNIOR LEAGUE 2024 - TANGA BOYS   
8  2024-05-26      2321        TCA DAR 30 OVERS CUP 2024 - DIV B   
9  2024-05-26      2322         TCA DAR 50 OVERS CUP 2024 -DIV A   
10 2024-05-26      2323         TCA DAR 50 OVERS CUP 2024 -DIV A   
11 2024-05-26      2324      TCA JUNIOR LEAGUE 2024 - GAIRO BOYS   
12 2024-05-26      2325        TCA DAR 30 OVERS CUP 2024 - DIV B   
13 2024-05-26      2326   TCA JUNIOR LEAGUE 2024

In [ ]:
# Ensure 'dates' column is in datetime format
new_df['dates'] = pd.to_datetime(new_df['dates'])

# Convert time columns to datetime by concatenating with dates
new_df['1st Inns Start'] = pd.to_datetime(new_df['dates'].dt.date.astype(str) + ' ' + new_df['1st Inns Start'].astype(str))
new_df['1st Inns End'] = pd.to_datetime(new_df['dates'].dt.date.astype(str) + ' ' + new_df['1st Inns End'].astype(str))
new_df['2nd Inns Start'] = pd.to_datetime(new_df['dates'].dt.date.astype(str) + ' ' + new_df['2nd Inns Start'].astype(str))
new_df['2nd Inns End'] = pd.to_datetime(new_df['dates'].dt.date.astype(str) + ' ' + new_df['2nd Inns End'].astype(str))

# Group by Scorer and aggregate start and end times
scorer_matches = new_df.groupby('Scorer').agg({'match_id': list, 'Fixture': list, '1st Inns Start': list, '1st Inns End': list, '2nd Inns Start': list, '2nd Inns End': list}).reset_index()

# Add a new column to check if each match of the scorer was played in a unique time
scorer_matches['Unique Time'] = scorer_matches.apply(lambda row: [
    all(
        (row['1st Inns Start'][i] >= row['2nd Inns End'][j] or row['2nd Inns Start'][j] >= row['1st Inns End'][i])
        for j in range(i)
    )
    for i in range(len(row['1st Inns Start']))
], axis=1)

# Identify matches that are not unique due to overlapping times
for i, row in scorer_matches.iterrows():
    for j in range(len(row['1st Inns Start'])):
        for k in range(j + 1, len(row['1st Inns Start'])):
            if (
                (row['1st Inns Start'][j] < row['2nd Inns End'][k] and row['1st Inns End'][j] > row['2nd Inns Start'][k])
                or
                (row['1st Inns Start'][k] < row['2nd Inns End'][j] and row['1st Inns End'][k] > row['2nd Inns Start'][j])
            ):
                scorer_matches.at[i, 'Unique Time'][j] = False
                scorer_matches.at[i, 'Unique Time'][k] = False

# Add a new column to note simultaneous matches
scorer_matches['Simul Matches'] = scorer_matches.apply(lambda row: [i for i, val in enumerate(row['Unique Time']) if not val], axis=1)

print(scorer_matches.head())

In [ ]:
# Initialize an empty list to store the extracted information
extracted_data2 = []

# Iterate over each row in the scorer_matches DataFrame
for idx, row in scorer_matches.iterrows():
    # Iterate over the Unique Time list to check if it's False
    for i, unique_time in enumerate(row['Unique Time']):
        if not unique_time:
            # Extract the information for the current index
            match_ids = row['match_id']
            for j in range(i):
                if not row['Unique Time'][j]:
                    extracted_data2.append({'match_id_1': match_ids[i], 'match_id_2': match_ids[j]})

# Create a DataFrame from the extracted data
clashing_pairs = pd.DataFrame(extracted_data2)

# Initialize an empty list to store the extracted information
extracted_data = []

# Iterate over each row in the scorer_matches DataFrame
for idx, row in scorer_matches.iterrows():
    # Iterate over the Unique Time list to check if it's False
    for i, unique_time in enumerate(row['Unique Time']):
        if not unique_time:
            # Extract the information for the current index
            scorer = row['Scorer']
            fixture = row['Fixture'][i]
            start_time = row['1st Inns Start'][i]
            end_time = row['2nd Inns End'][i]
            match_id = row['match_id'][i]
            
            # Append the extracted information to the list
            extracted_data.append({'Scorer': scorer, 'Fixture': fixture, '1st Inns Start': start_time, '2nd Inns End': end_time, 'match_id': match_id})

# Create a DataFrame from the extracted data
simul_matches = pd.DataFrame(extracted_data)

print(simul_matches)
print(clashing_pairs)

In [ ]:
# Initialize an empty list to store the values of the "Clashing With" column
clashing_with_list = []

# Iterate over each row in the simul_matches DataFrame
for idx, row in simul_matches.iterrows():
    match_id = row['match_id']
    
    # Check if the match_id exists in the clashing_pairs DataFrame
    if match_id in clashing_pairs['match_id_1'].values:
        # Get the corresponding value from the 'match_id_2' column
        clashing_with = clashing_pairs.loc[clashing_pairs['match_id_1'] == match_id, 'match_id_2'].iloc[0]
    elif match_id in clashing_pairs['match_id_2'].values:
        # Get the corresponding value from the 'match_id_1' column
        clashing_with = clashing_pairs.loc[clashing_pairs['match_id_2'] == match_id, 'match_id_1'].iloc[0]
    else:
        clashing_with = None
    
    # Append the clashing_with value to the list
    clashing_with_list.append(clashing_with)

# Add the list as a new column to the simul_matches DataFrame
simul_matches['Clashing With'] = clashing_with_list

# Drop columns except 'Scorer', 'match_id', and 'Clashing With'
simul_matches = simul_matches[['match_id', 'Clashing With']]

print(simul_matches)

In [ ]:
# Merge the two DataFrames on the 'match_id' column
merged_df = pd.merge(new_df, simul_matches, on='match_id', how='outer')

print(merged_df)

In [14]:
# Selecting desired columns and renaming them
audit_df = new_df.loc[new_df['Doubt Level'] == 'High']

audit_df = audit_df[['dates', 'event_name', 'Fixture', 'first_bat_overs', 'second_bat_overs', 'Exact Problem', 'Scorer']]
audit_df.columns = ['Date', 'Competition', 'Fixture', '1st Inns Overs', '2nd Inns Overs', 'Exact Problems', 'Scorer']
audit_df

,Date,Competition,Fixture,1st Inns Overs,2nd Inns Overs,Exact Problems,Scorer
1,2024-05-24,TCA JUNIOR LEAGUE 2024 - CHAMWINO GIRLS,Chamwino Secondary A Girls vs Mkapa C Girls,20.0,13.1,1. 2nd Innings over rate was too slow: 6.61 mi...,Ally Mpeka ...
7,2024-05-25,TCA JUNIOR LEAGUE 2024 - TANGA BOYS,Gofu Juu Boys vs Kwanjeka Boys,7.5,3.2,2nd Innings over rate was too quick: 1.2 minut...,Sefu Athuman ...
9,2024-05-26,TCA DAR 50 OVERS CUP 2024 -DIV A,Delaware Upanga SC - A vs Caravans,42.3,33.0,2nd Innings over rate was too slow: 6.06 minut...,Neema Justine ...
10,2024-05-26,TCA DAR 50 OVERS CUP 2024 -DIV A,Pak Stars A vs TCA Combine,50.0,45.5,Innings break was too long: 60 minutes long.,Irene Kasembe ...
15,2024-05-26,TCA JUNIOR LEAGUE 2024 - GAIRO BOYS,Lukungu Boys vs Msingisi Boys,16.3,8.5,1. 2nd Innings over rate was too quick: 2.49 m...,Sefu Khalifa ...


In [15]:
export_folder = "F:\\1. Project\\TCA Analyst\Projects\\11. Junior League Scoring Audits"
audit_df.to_excel(f"{export_folder}/Junior League Scoring Analysis.xlsx", index=False)

### Series Info Scraping

In [211]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Initialize an empty DataFrame
series_df = pd.DataFrame()

series_id = 1
no_info_count = 0
max_fail_count = 5

while no_info_count < max_fail_count:
    url = f'https://www.cricclubs.com/Tanzania/viewLeague.do?league={series_id}&clubId=7605'
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Connection failed for series ID {series_id}")
        series_id += 1
        continue

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the score-board div
    scoreboard_div = soup.find('div', class_='score-board')

    # Check if scoreboard_div is found
    if scoreboard_div is None:
        print(f"No information found for series ID {series_id}")
        no_info_count += 1
        series_id += 1
        continue

    # Reset no_info_count if information is found
    no_info_count = 0

    # Extract series details if available
    series_details_tag = scoreboard_div.find('h3', class_='theme-color')
    series_details = series_details_tag.text if series_details_tag else None

    columns = []
    rows = []

    columns.append("event_name")
    rows.append(series_details)

    # Find all tables inside the score-board div
    tables = scoreboard_div.find_all('table')

    # Initialize data list
    data = []

    # Extract data rows from all tables
    for table in tables:
        for row in table.find_all('tr'):
            row_data = [td.text.strip() for td in row.find_all('td')]
            if row_data:
                data.append(row_data)

    for row in data[:4]:  # Process only the first 4 items
        columns.append(row[0])
        rows.append(row[2])
        if len(row) > 3:
            columns.append(row[3])
            rows.append(row[5])

    # Create DataFrame for the current series ID
    df = pd.DataFrame([rows], columns=columns)

    # Append the DataFrame to the final DataFrame
    series_df = pd.concat([series_df, df], ignore_index=True)

    series_id += 1

print(series_df.head())

No information found for series ID 13
No information found for series ID 14
No information found for series ID 24
No information found for series ID 34
No information found for series ID 38
No information found for series ID 39
No information found for series ID 40
No information found for series ID 108
No information found for series ID 118
No information found for series ID 134
No information found for series ID 150
No information found for series ID 151
No information found for series ID 152
No information found for series ID 153
No information found for series ID 154
                        event_name  Start Date Category     Ball Type Level  \
0           DRCC CARAVANS CUP 2018  21/10/2018      Men  Leather Ball  Club   
1  DRCC AFRO TURK PRO10 DIV-A 2019  13/01/2019      Men  Leather Ball  Club   
2           DRCC PRO-10 DIV-B 2019  17/03/2019      Men  Leather Ball  Club   
3           DRCC PRO-10 DIV-C 2019  17/03/2019      Men  Leather Ball  Club   
4           DRCC PRO-10 DIV

In [212]:
series_df.to_excel("series_info.xlsx", index=False)

## PLAYERS INFO

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to scrape player names and ids from a given team URL
def scrape_team_players(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    players_data = []

    players_div = soup.find('div', id='allPlayersDiv')
    if not players_div:
        return players_data

    player_cards = players_div.find_all('div', class_='col-sm-3 col-xs-6')

    for player_card in player_cards:
        player_name = player_card.get('id')
        player_profile_link = player_card.find('a', class_='btn btn-team')['href']
        player_id = player_profile_link.split('playerId=')[1].split('&')[0]
        players_data.append({
            'Player Name': player_name,
            'Player ID': player_id
        })

    return players_data

# Main function to cycle through team URLs and compile player data
def main():
    # Example URLs (you'll need to update this with actual team URLs)
    base_url = 'https://www.cricclubs.com/Tanzania/viewTeam.do?teamId={}&league=152&clubId=7605'
    team_ids = range(0, 1470)  # Example range of team IDs

    all_players_data = []

    for team_id in team_ids:
        team_url = base_url.format(team_id)
        team_players = scrape_team_players(team_url)
        all_players_data.extend(team_players)

    # Create a DataFrame from the collected data
    df = pd.DataFrame(all_players_data)
    return df

# Run the main function and display the DataFrame
player_ids_df = main()
player_ids_df.head()  # Display the first few rows of the DataFrame


,Player Name,Player ID
0,Abdallah Bakari,1844975
1,Abdulqadir Jivajee,1844977
2,Abubakari Abdallah,1844967
3,Athumani Yusuph,1924909
4,Bilali Hamadi,1844976


In [4]:
player_ids_df.describe()

,Player Name,Player ID
count,24855,24855
unique,9498,10058
top,Azhaar Premji,1031220
freq,45,45


In [5]:
player_ids_df = player_ids_df.drop_duplicates()
player_ids_df.describe()

,Player Name,Player ID
count,10058,10058
unique,9498,10058
top,Ismail Ibrahim,1844975
freq,5,1


In [8]:
trial_players = player_ids_df.tail()

In [15]:
# Function to scrape data for a given chunk of player IDs
def scrape_player_data(chunk_df, chunk_index):
    players_data = []

    for index, row in chunk_df.iterrows():
        player_id = row['Player ID']

        url = f'https://www.cricclubs.com/Tanzania/viewPlayer.do?playerId={player_id}&clubId=7605'
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code != 200:
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        # Try to extract the player's name
        h4_tag = soup.find('h4')
        if not h4_tag:
            continue

        player_name = h4_tag.find('span').text.strip()

        # Extract the other information
        info = {
            'Player': player_name,
            'CC Player ID': player_id,
            'Current Team': None,
            'Teams': None,
            'Playing Role': None,
            'Batting Style': None,
            'Bowling Style': None
        }

        # Loop through all <p> tags and extract the data
        for p in soup.find_all('p'):
            text = p.get_text(separator=' ').strip()
            if 'CC Player ID' in text:
                info['CC Player ID'] = text.split(':')[1].strip()
            elif 'Current Team' in text:
                a_tag = p.find('a')
                if a_tag:
                    info['Current Team'] = a_tag.text.strip()
            elif 'Teams' in text:
                info['Teams'] = text.split(':')[1].strip()
            elif 'Playing Role' in text:
                strong_tag = p.find('strong')
                if strong_tag:
                    info['Playing Role'] = strong_tag.text.strip()
            elif 'Batting Style' in text:
                strong_tag = p.find('strong')
                if strong_tag:
                    info['Batting Style'] = strong_tag.text.strip()
            elif 'Bowling Style' in text:
                strong_tag = p.find('strong')
                if strong_tag:
                    info['Bowling Style'] = strong_tag.text.strip()

        # Append the player's data to the list
        players_data.append(info)
        # print(f"Successfully scraped data for player ID: {player_id}")

    # Create a DataFrame from the list of dictionaries
    players_df = pd.DataFrame(players_data)

    # Save the DataFrame to a file
    filename = f'players_data_chunk_{chunk_index}.csv'
    players_df.to_csv(filename, index=False)
    print(f'Saved chunk {chunk_index} to {filename}')

# Split the player_ids_df into chunks of 1000
chunk_size = 1000
num_chunks = (len(player_ids_df) + chunk_size - 1) // chunk_size  # Calculate the number of chunks

for i in range(num_chunks):
    chunk_df = player_ids_df[i * chunk_size:(i + 1) * chunk_size]
    scrape_player_data(chunk_df, i)

Saved chunk 0 to players_data_chunk_0.csv
Saved chunk 1 to players_data_chunk_1.csv
Saved chunk 2 to players_data_chunk_2.csv
Saved chunk 3 to players_data_chunk_3.csv
Saved chunk 4 to players_data_chunk_4.csv
Saved chunk 5 to players_data_chunk_5.csv
Saved chunk 6 to players_data_chunk_6.csv
Saved chunk 7 to players_data_chunk_7.csv
Saved chunk 8 to players_data_chunk_8.csv
Saved chunk 9 to players_data_chunk_9.csv
Saved chunk 10 to players_data_chunk_10.csv


In [16]:
# List to hold DataFrames
dfs = []

# Number of chunks
num_chunks = 11

# Read each chunk and append to the list
for i in range(num_chunks):
    filename = f'players_data_chunk_{i}.csv'
    df = pd.read_csv(filename)
    dfs.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

# Export the combined DataFrame to an Excel file
combined_df.to_excel('combined_players_data.xlsx', index=False)

print('All chunks have been merged and exported to combined_players_data.xlsx')

All chunks have been merged and exported to combined_players_data.xlsx


## Others

### Converting Raw Data into a DF

In [ ]:
import re
import pandas as pd

# Initialize lists to store data
over = []
ball = []
batter = []
bowler = []
event = []
other_event = []
commentary = []
wickets = []

# Iterate through each content and extract information
for content in content_list:
    # Split the content by comma
    parts = content.split(",")
    
    # Extract over and ball
    over_ball = parts[0].split(".")
    over.append(int(over_ball[0]) + 1)
    # Get the single digit immediately after the decimal point
    ball.append(over_ball[1][0])
    event.append(over_ball[1][1])
    other_event_char = ''.join(filter(str.islower, over_ball[1][2]))  # Extract lowercase characters
    other_event.append(other_event_char)
    
    # Extract batter, bowler, and event
    batter_bowler_event = re.split(r'(?<=[a-zA-Z])(?=[A-Z])', parts[0].split("to")[0].strip())
    # Remove the first capitalized letter if two are detected in a row
    if len(batter_bowler_event) > 1 and batter_bowler_event[0].isupper() and batter_bowler_event[1].isupper():
        batter_bowler_event = batter_bowler_event[1:]
    bowler.append(''.join(filter(str.isalpha, batter_bowler_event[-1])))  # Extract only alphabets
    batter.append(''.join(filter(str.isalpha, parts[0].split("to")[1].strip())))  # Extract only alphabets
        
    if len(parts) > 2 and parts[2] is not None:
        commentary_text = parts[2].strip()
        wickets.append(1)
        # Remove "OUT" from the commentary
        commentary_text = commentary_text.replace("OUT", "")
        # Split commentary by "(" and take the first part
        commentary_split = commentary_text.split("(")[0].strip()
        # Remove any remaining numbers
        commentary_cleaned = re.sub(r'\d+', '', commentary_split)
        commentary.append(commentary_cleaned)
    elif len(parts) <= 2:
        commentary_text = parts[1].strip()
        if "OUT" in commentary_text:  # Include only those with "OUT"
            wickets.append(1)
            # Remove "OUT" from the commentary
            commentary_text = commentary_text.replace("OUT", "")
            # Split commentary by "(" and take the first part
            commentary_split = commentary_text.split("(")[0].strip()
            # Remove any remaining numbers
            commentary_cleaned = re.sub(r'\d+', '', commentary_split)
            commentary.append(commentary_cleaned)
        else:
            commentary.append(None)  # If not "OUT", insert None
            wickets.append(0)

# Check if all lists have the same length
assert len(over) == len(ball) == len(batter) == len(bowler) == len(event) == len(other_event) == len(commentary) == len(wickets), "Length mismatch in lists" 
        
# Create a DataFrame
data = {
    "over": over,
    "delivery": ball,
    "batter": batter,
    "bowler": bowler,
    "Event": event,
    "Other_Event": other_event,
    "Commentary": commentary,
    "wickets": wickets
}

df = pd.DataFrame(data)

# Create bowler_wicket column based on the presence of "b" in the Commentary
df['bowler_wicket'] = df['Commentary'].apply(lambda x: 1 if 'b' in str(x) else 0)

# Create How_Out column based on the presence of different types of dismissals
def determine_how_out(comment):
    if 'c & b' in str(comment):
        return 'caught and bowled'
    elif 'c' in str(comment):
        return 'caught'
    elif 'lbw' in str(comment):
        return 'lbw'
    elif 'run out' in str(comment):
        return 'run out'
    elif 'b' in str(comment):
        return 'bowled'
    else:
        return None

df['wicket_type'] = df['Commentary'].apply(determine_how_out)

# Extract first 2 names from the Commentary and append to Batter_Out column
df['player_out'] = df['Commentary'].str.extract(r'([A-Z][a-z]* [A-Z][a-z]*)')

# Map the 'other_event' column to corresponding 'extras type'
extras_mapping = {
    'w': 'wides',
    'n': 'noballs',
    'l': 'legbyes',
    'b': 'byes'
}
df['extras_type'] = df['Other_Event'].map(extras_mapping)

def extract_runs(other_event, event):
    if other_event == 'n' and event.isdigit():
        event_num = int(event)
        if event_num > 1:
            return event_num - 1, 1
        else:
            return 0, 1
    elif other_event in ['w', 'l', 'b'] and event.isdigit():
        return 0, int(event)
    elif event.isdigit():
        return int(event), 0
    else:
        return 0, 0

# Apply the function to the DataFrame
df['batter_runs'], df['extras_runs'] = zip(*df.apply(lambda x: extract_runs(x['Other_Event'], x['Event']), axis=1))

# Create Total_Runs column based on the value of 'Event'
df['total_runs'] = df['Event'].apply(lambda x: int(x) if x.isdigit() else 0)

# Print the DataFrame
print(df)

In [ ]:
df.drop(columns=['Event', 'Other_Event', 'Commentary'], inplace=True)
df.tail(50)

In [ ]:
df["match_number"] = 16
df["inning_number"] = 1
df['batting_team'] = 'Namibia'
df['bowling_team'] = 'Zimbabwe'
df['event_name'] = "African Games"
df

In [ ]:
df.to_csv("zim_nam_1.csv", index=False)

### Combining All CSV files in the folder into 1

In [ ]:
import os
import pandas as pd

# Get the current directory
directory = os.getcwd()

# Initialize an empty DataFrame to store the concatenated data
merged_df = pd.DataFrame()

# Iterate over each file in the directory
for file in os.listdir(directory):
    if file.endswith(".csv"):  # Check if the file is a CSV file
        file_path = os.path.join(directory, file)  # Get the full file path
        df = pd.read_csv(file_path)  # Read the CSV file into a DataFrame
        merged_df = pd.concat([merged_df, df], ignore_index=True)  # Concatenate the DataFrame

# Display the merged DataFrame
print(merged_df)

In [ ]:
merged_df.to_csv("march_t20s_men_innings_data.csv", index=False)

In [ ]:
african_games = merged_df.loc[merged_df.event_name == "African Games"]
african_games

In [ ]:
african_games.to_csv("african_games_innings_data.csv", index=False)

### FAILED FEATURE ADDITIONS

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Initialize the WebDriver
driver = webdriver.Chrome()

# Load the URL
driver.get(url)

# Find the span element containing the desired information
information_span = driver.find_element(By.XPATH, "//span[@class='ds-text-tight-s ds-font-regular ds-text-typo']")

# Replace the text with "KUW"
if information_span:
    driver.execute_script("arguments[0].innerText = 'KUW';", information_span)
    
# Wait for 5 seconds
time.sleep(5)

# Close the WebDriver
driver.quit()

In [ ]:
from selenium import webdriver

# Initialize the WebDriver
driver = webdriver.Chrome()

# Load the HTML content
driver.get(url)

# Find the span element containing the desired information
information_span = driver.find_element(By.XPATH, "//span[@class='ds-text-tight-s ds-font-regular ds-text-typo']")

# Extract the text from the span element
info_text = information_span.text.strip() if information_span else None

print(info_text)

# Close the WebDriver
driver.quit()

In [ ]:
ds-text-tight-m ds-font-regular ds-flex ds-px-3 ds-py-2 lg:ds-px-4 lg:ds-py-[10px] ds-items-start ds-select-none lg:ds-select-auto
ds-text-tight-m ds-font-regular ds-flex ds-px-3 ds-py-2 lg:ds-px-4 lg:ds-py-[10px] ds-items-start ds-select-none lg:ds-select-auto
ds-text-tight-m ds-font-regular ds-flex ds-px-3 ds-py-2 lg:ds-px-4 lg:ds-py-[10px] ds-items-start ds-select-none lg:ds-select-auto ds-items-center

In [ ]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the website
url = 'https://www.espncricinfo.com/series/icc-cricket-world-cup-challenge-league-play-off-2023-24-1420549/italy-vs-tanzania-18th-match-super-sixes-1420585/ball-by-ball-commentary'
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all div elements with either of the specified classes
target_divs = soup.find_all('div', class_=['ds-text-tight-m ds-font-regular ds-flex ds-px-3 ds-py-2 lg:ds-px-4 lg:ds-py-[10px] ds-items-start ds-select-none lg:ds-select-auto', 'ds-text-tight-m ds-font-regular ds-flex ds-px-3 ds-py-2 lg:ds-px-4 lg:ds-py-[10px] ds-items-start ds-select-none lg:ds-select-auto ds-items-center'])

# Extract content from the divs
for div in target_divs:
    # Extract content as needed
    content = div.text.strip()
    print(content)